In [1]:
using Distributions
using Interpolations
using CSV
using JLD

main_path = "/home/artur/BondPricing"
module_path = string(main_path, "/", "Julia/modules/")
modls = ["ModelObj", "AnalyticFunctions", 
         "BondPrInterp", "EqFinDiff", "Batch"]
for modl in modls
    include(string(joinpath(module_path, modl), "/", modl, ".jl"))
end
include(string(module_path, "/", "TestFunctions.jl"))

Main.TestFunctions

In [2]:
bt = Batch.BatchObj()
comb_num = 2
bt = Batch.set_par_dict(bt, comb_num)

# Adjust parameters
bt.mi._svm_dict["lambda"] = .3
bt.mi._svm_dict["sigmal"] = .2
bt.mi._svm_dict["sigmah"] = .2
bt.mi._svm_dict["kappa"] = 0.015

# Set Directories & Paths (Main, Batch, Maturity, Combination)
bt = Batch.set_comb_res_paths(bt)

# Create Directories
bt = Batch.mk_comb_res_dirs(bt)

# Construct Firm Object
svm = ModelObj.firm_constructor(bt.mi._svm_dict)

# Bond Pricing Interpolation - Inputs
# svm.bi.vtN = 12
# svm.bi.ttmN = 10
# svm.bi.vbhlN = 11 

# Set Bond Pricing Variables' Grids
svm = ModelObj.set_bpr_grids(svm)

Setting initial vbl value to NaN
Setting initial vbh value to NaN
Setting initial c value to NaN
Setting initial p value to NaN


Main.ModelObj.Firm
  mu_b: Float64 1.0
  m: Float64 1.0
  c: Float64 NaN
  p: Float64 NaN
  vbl: Float64 NaN
  vbh: Float64 NaN
  pm: Main.ModelObj.FirmParams
  bi: Main.ModelObj.BPrInputs
  bs: Main.ModelObj.BPrSurfs
  bf: Main.ModelObj.BPrInterpFuns


In [40]:
# Compute Bond Price Inputs
svm = @time BondPrInterp.bpr_surfs(svm)

966.326982 seconds (26.01 M allocations: 21.848 GiB, 0.63% gc time)


Main.ModelObj.Firm
  mu_b: Float64 1.0
  m: Float64 1.0
  c: Float64 NaN
  p: Float64 NaN
  vbl: Float64 NaN
  vbh: Float64 NaN
  pm: Main.ModelObj.FirmParams
  bi: Main.ModelObj.BPrInputs
  bs: Main.ModelObj.BPrSurfs
  bf: Main.ModelObj.BPrInterpFuns


In [315]:
# Interpolate Bond Pricing Surfaces
#  svm = @time BondPrInterp.bpr_interp(svm)

  0.271363 seconds (905.58 k allocations: 141.655 MiB, 8.98% gc time)


Main.ModelObj.Firm
  mu_b: Float64 1.0
  m: Float64 1.0
  c: Float64 NaN
  p: Float64 NaN
  vbl: Float64 NaN
  vbh: Float64 NaN
  pm: Main.ModelObj.FirmParams
  bi: Main.ModelObj.BPrInputs
  bs: Main.ModelObj.BPrSurfs
  bf: Main.ModelObj.BPrInterpFuns


# Save and Load Results

Obs: JLD cannot handle interpolated functions.  Need to interpolate after loading results.

In [41]:
Batch.save_batch_results(bt, svm=svm)

In [3]:
res = Batch.load_batch_results(bt)

Dict{String,Any} with 2 entries:
  "bt"  => BatchStruct(DirFileNames("Julia", "Results/", "m_", "c_", "debt_at_p…
  "svm" => Firm(1.0, 1.0, NaN, NaN, NaN, NaN, FirmParams(100.0, 0.6, 0.27, 0.08…

In [4]:
svm2 = res["svm"]

Main.ModelObj.Firm
  mu_b: Float64 1.0
  m: Float64 1.0
  c: Float64 NaN
  p: Float64 NaN
  vbl: Float64 NaN
  vbh: Float64 NaN
  pm: Main.ModelObj.FirmParams
  bi: Main.ModelObj.BPrInputs
  bs: Main.ModelObj.BPrSurfs
  bf: Main.ModelObj.BPrInterpFuns


In [5]:
# Interpolate Bond Pricing Surfaces
svm2 = @time BondPrInterp.bpr_interp(svm2)

  5.697847 seconds (14.45 M allocations: 822.295 MiB, 7.10% gc time)


Main.ModelObj.Firm
  mu_b: Float64 1.0
  m: Float64 1.0
  c: Float64 NaN
  p: Float64 NaN
  vbl: Float64 NaN
  vbh: Float64 NaN
  pm: Main.ModelObj.FirmParams
  bi: Main.ModelObj.BPrInputs
  bs: Main.ModelObj.BPrSurfs
  bf: Main.ModelObj.BPrInterpFuns


In [ ]:
bt3 = Batch.load_batch_results(bt)

In [45]:
c = 4.
p = 42.10472
vbl =AnalyticFunctions.get_cvm_vb(svm, svm.pm.sigmal; c=c, p=p)
ttm =  0.99999
Vt=vbl * exp(.2)
BondPrInterp.get_svm_bond_price(svm2, vbl, ttm; Vt=Vt, c=c, p=p) - 
BondPrInterp.get_cvm_bond_price(svm2, ttm, svm.pm.sigmal, Vt=Vt, c=c, p=p)


-0.002666945994334924

# Homogeneity in (1/$\mu_b$, m, c, p)

In [338]:
using Distributed

phi = 1/3.
mu_b = 1.
cgrid = range(1, 6, length=6) 
pgrid = range(40, 70, length=5)

40.0:7.5:70.0

In [329]:
# Bond Pricing Function is Homogenous of Degree 1 in (1/mu_b, c, p):
fun="bond"
cvm_bpr_diffs = fetch(@spawn [TestFunctions.homogeneity_tests(svm, phi, c, p; 
                                                              model="cvm", fun=fun) 
                        for c in cgrid, p in pgrid])

println("Maximum ", fun, " difference: ", maximum(cvm_bpr_diffs))
cvm_bpr_diffs

Maximum bond difference: 1.4210854715202004e-14


6×5 Array{Float64,2}:
  0.0          7.10543e-15  -7.10543e-15  0.0           0.0        
  7.10543e-15  7.10543e-15   7.10543e-15  7.10543e-15  -1.42109e-14
  0.0          0.0           7.10543e-15  0.0          -7.10543e-15
 -7.10543e-15  0.0           0.0          1.42109e-14  -7.10543e-15
  7.10543e-15  7.10543e-15   0.0          7.10543e-15   0.0        
  0.0          7.10543e-15   7.10543e-15  0.0           0.0        

In [331]:
fun="bond"
svm_bpr_diffs = fetch(@spawn [TestFunctions.homogeneity_tests(svm, phi, c, p; 
                                                              model="svm", fun=fun) 
                        for c in cgrid, p in pgrid])

println("Maximum ", fun, " difference: ", maximum(svm_bpr_diffs))
svm_bpr_diffs

Maximum bond difference: 1.4210854715202004e-14


6×5 Array{Float64,2}:
 7.10543e-15  0.0           0.0          7.10543e-15   7.10543e-15
 0.0          7.10543e-15  -7.10543e-15  0.0           0.0        
 0.0          0.0           0.0          1.42109e-14   7.10543e-15
 7.10543e-15  0.0           0.0          7.10543e-15  -7.10543e-15
 7.10543e-15  1.42109e-14   7.10543e-15  7.10543e-15   0.0        
 0.0          7.10543e-15   7.10543e-15  0.0           1.42109e-14

In [332]:
fun="vb"
cvm_vb_diffs = fetch(@spawn [TestFunctions.homogeneity_tests(svm, phi, c, p; 
                                                             model="cvm", fun=fun) 
                        for c in cgrid, p in pgrid])

println("Maximum ", fun, " difference: ", maximum(cvm_vb_diffs))
cvm_vb_diffs

Maximum vb difference: 2.842170943040401e-14


6×5 Array{Float64,2}:
 7.10543e-15   0.0          0.0          -1.42109e-14  1.42109e-14
 0.0          -1.42109e-14  1.42109e-14   0.0          2.84217e-14
 7.10543e-15   0.0          7.10543e-15   0.0          1.42109e-14
 7.10543e-15   0.0          0.0           0.0          2.84217e-14
 0.0           0.0          0.0           0.0          0.0        
 7.10543e-15   1.42109e-14  0.0           1.42109e-14  0.0        

In [339]:
fun="debt"
cvm_debt_diffs = fetch(@spawn [TestFunctions.homogeneity_tests(svm2, phi, c, p; 
                                                               model="cvm", fun=fun) 
                        for c in cgrid, p in pgrid])

println("Maximum ", fun, " difference: ", maximum(cvm_debt_diffs))
cvm_debt_diffs

Maximum debt difference: 2.842170943040401e-14


6×5 Array{Float64,2}:
  7.10543e-15  7.10543e-15  -1.42109e-14  1.42109e-14  0.0        
 -7.10543e-15  0.0           2.84217e-14  0.0          1.42109e-14
  0.0          7.10543e-15   0.0          0.0          1.42109e-14
  0.0          1.42109e-14   1.42109e-14  1.42109e-14  0.0        
  0.0          7.10543e-15  -1.42109e-14  0.0          1.42109e-14
  0.0          7.10543e-15   0.0          7.10543e-15  0.0        

In [340]:
fun="debt"
svm_debt_diffs = fetch(@spawn [TestFunctions.homogeneity_tests(svm2, phi, c, p; 
                                                               model="svm", fun=fun)
                        for c in cgrid, p in pgrid])

println("Maximum ", fun, " difference: ", maximum(svm_debt_diffs))
svm_debt_diffs

Maximum debt difference: 1.5063505998114124e-12


6×5 Array{Float64,2}:
  1.42109e-13  -7.10543e-15   8.73968e-13   1.09424e-12   3.69482e-13
  2.20268e-13  -4.83169e-13   1.84741e-13   7.17648e-13  -2.00373e-12
 -7.10543e-13  -3.19744e-13   4.76064e-13  -2.20268e-13   1.16529e-12
  2.84217e-14  -2.55795e-13  -3.97904e-13  -2.4869e-13   -2.10321e-12
 -4.9738e-13   -3.76588e-13   1.84741e-13  -2.4869e-13   -1.49214e-12
  4.26326e-14   5.04485e-13   7.31859e-13   1.50635e-12   9.66338e-13

In [341]:
fun="equity"
cvm_eqs = fetch(@spawn [EqFinDiff.get_cvm_eq(svm2, svm.pm.V0, svm.pm.sigmal; 
                                                  mu_b=mu_b, c=c, p=p)
                        for c in cgrid, p in pgrid])
cvm_eqs

6×5 Array{Float64,2}:
 56.2356  46.4703  35.9492  24.6821  13.3938
 58.8428  48.9255  38.1792  26.5796  14.805 
 61.4699  51.4103  40.4513  28.5345  16.2885
 64.1159  53.9236  42.7641  30.5453  17.8434
 66.7798  56.4642  45.1161  32.6105  19.4688
 69.4608  59.031   47.506   34.7284  21.1636

In [342]:
fun="equity"
cvm_eq_diffs = fetch(@spawn [TestFunctions.homogeneity_tests(svm2, phi, c, p; 
                                                             model="cvm", fun=fun) 
                        for c in cgrid, p in pgrid])

println("Maximum ", fun, " difference: ", maximum(cvm_eq_diffs))
cvm_eq_diffs

Maximum equity difference: 2.842170943040401e-14


6×5 Array{Float64,2}:
 -7.10543e-15   0.0           2.13163e-14  -6.39488e-14  -1.74083e-13
  0.0           7.10543e-15   7.10543e-15   1.06581e-14   2.84217e-14
 -7.10543e-15  -1.42109e-14  -7.10543e-15  -2.13163e-14  -5.32907e-14
  0.0           2.13163e-14  -2.84217e-14   1.06581e-14  -7.10543e-15
  0.0          -2.13163e-14   0.0           7.10543e-15  -2.13163e-14
  0.0           2.84217e-14  -7.10543e-15  -2.84217e-14   7.10543e-15

In [345]:
fun="equity"
svm_eq_diffs = fetch(@spawn [TestFunctions.homogeneity_tests(svm2, phi, c, p; 
            model="svm", fun=fun) 
                        for c in cgrid, p in pgrid])

println("Maximum ", fun, " difference: ", maximum(svm_eq_diffs))
svm_eq_diffs

Computing Equity Vmax
Vmax: 125.0
phi0: -42.51391951978036
phi1: 1.0
pv_rfdebt: 38.64330013215303
debt_abs_per_diff: 2.657196132952841e-5
eq_abs_per_diff: 0.0009688543375786891
Equity Vmax: 193.91602699731453
 
here
eq_max: 151.255420888986
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.010124921798706055
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 56.236411313504114
Equity Core Function Computation Time: 0.03593611717224121
Total Equity FD Core Function Computation Time: 0.0975959300994873
Total computation time: 0.11720800399780273
Computing Equity Vmax
Vmax: 125.0
phi0: -42.5139195197804
phi1: 1.0
pv_rfdebt: 38.64330013215303
debt_abs_per_diff: 2.657196132952841e-5
eq_abs_per_diff: 0.0009688543375779385
Equity Vmax: 193.91602699731453
 
here
eq_max: 151.25542088898598
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Con

equity: 46.47077447203721
Equity Core Function Computation Time: 0.039598941802978516
Total Equity FD Core Function Computation Time: 0.09024405479431152
Total computation time: 0.11508488655090332
Computing Equity Vmax
Vmax: 125.0
phi0: -51.010207001660206
phi1: 1.0
pv_rfdebt: 45.79806846780551
debt_abs_per_diff: 3.4887099767274305e-6
eq_abs_per_diff: 0.0008520455352143843
Equity Vmax: 235.70614279040458
 
here
eq_max: 184.53856644128334
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.010137081146240234
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 46.47077447203721
Equity Core Function Computation Time: 0.03941917419433594
Total Equity FD Core Function Computation Time: 0.09000802040100098
Total computation time: 0.1147918701171875
Computing Equity Vmax
Vmax: 125.0
phi0: -48.21059328474844
phi1: 1.0
pv_rfdebt: 46.2826041431453
debt_abs_per_diff: 2.4

equity: 35.94806099454654
Equity Core Function Computation Time: 0.039868831634521484
Total Equity FD Core Function Computation Time: 0.09055709838867188
Total computation time: 0.11934709548950195
Computing Equity Vmax
Vmax: 125.0
phi0: -56.70688076662812
phi1: 1.0
pv_rfdebt: 53.43737247879778
debt_abs_per_diff: 2.8775236726454635e-6
eq_abs_per_diff: 0.0009282489537589996
Equity Vmax: 272.8593235477421
 
here
eq_max: 215.95179950225
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.010154962539672852
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 38.177373875095526
Equity Core Function Computation Time: 0.039878129959106445
Total Equity FD Core Function Computation Time: 0.09058785438537598
Total computation time: 0.11937904357910156
Computing Equity Vmax
Vmax: 125.0
phi0: -56.7068807666282
phi1: 1.0
pv_rfdebt: 53.43737247879778
debt_abs_per_diff: 2.877

equity: 26.577094652941543
Equity Core Function Computation Time: 0.03978300094604492
Total Equity FD Core Function Computation Time: 0.09053802490234375
Total computation time: 0.12312793731689453
Computing Equity Vmax
Vmax: 125.0
phi0: -65.203168248508
phi1: 1.0
pv_rfdebt: 60.592140814450275
debt_abs_per_diff: 3.0996211018882616e-6
eq_abs_per_diff: 0.0008348860669948674
Equity Vmax: 315.86877442195504
 
here
eq_max: 250.456328951378
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.010143041610717773
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 26.577094652941376
Equity Core Function Computation Time: 0.03966498374938965
Total Equity FD Core Function Computation Time: 0.0904080867767334
Total computation time: 0.1230309009552002
Computing Equity Vmax
Vmax: 125.0
phi0: -62.403554531596114
phi1: 1.0
pv_rfdebt: 61.07667648979007
debt_abs_per_diff: 2.294

equity: 14.803232092095124
Equity Core Function Computation Time: 0.03947305679321289
Total Equity FD Core Function Computation Time: 0.09023594856262207
Total computation time: 0.1254899501800537
Computing Equity Vmax
Vmax: 125.0
phi0: -70.89984201347596
phi1: 1.0
pv_rfdebt: 68.23144482544255
debt_abs_per_diff: 2.5515967789317238e-6
eq_abs_per_diff: 0.0009461670828778451
Equity Vmax: 348.2453238002054
 
here
eq_max: 277.08306662127796
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.010141849517822266
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 16.285564615354417
Equity Core Function Computation Time: 0.03967094421386719
Total Equity FD Core Function Computation Time: 0.09048795700073242
Total computation time: 0.12572813034057617
Computing Equity Vmax
Vmax: 125.0
phi0: -70.899842013476
phi1: 1.0
pv_rfdebt: 68.23144482544255
debt_abs_per_diff: 2.551

6×5 Array{Float64,2}:
  4.26326e-14   0.0          -3.55271e-14  -9.23706e-14  -3.90799e-14
  0.0          -2.13163e-14   7.10543e-14   1.66978e-13   2.66454e-14
 -2.13163e-14   0.0           3.55271e-14   0.0          -1.27898e-13
  0.0           0.0          -2.84217e-14   7.81597e-14   1.81188e-13
  1.42109e-14  -1.42109e-14   2.84217e-14   0.0           1.1724e-13 
  0.0           7.81597e-14   0.0           0.0           1.35003e-13

# Compare CVM to SVM

In [189]:
main_path = "/home/artur/BondPricing"
module_path = string(main_path, "/", "Julia/modules/")
include(string(module_path, "/", "TestFunctions.jl"))
modls = ["Batch", "ModelObj", "AnalyticFunctions", 
         "BondPrInterp", "EqFinDiff"]
for modl in modls
    include(string(joinpath(module_path, modl), "/", modl, ".jl"))
end

LoadError: LoadError: syntax: "do" at /home/artur/BondPricing/Julia/modules/Batch/_batch_file_methods.jl:22 expected "end", got "else"
in expression starting at /home/artur/BondPricing/Julia/modules/Batch/_batch_file_methods.jl:24

In [349]:
cgrid = range(1, 6, length=6) 
pgrid = range(40, 70, length=5)

40.0:7.5:70.0

In [380]:
fun="bond"
bpr_diffs = fetch(@spawn [TestFunctions.cvm_svm_comparison(svm2, c, p; fun=fun) 
                        for c in cgrid, p in pgrid])

println("Maximum ", fun, " difference: ", maximum(bpr_diffs))
bpr_diffs

Maximum bond difference: 0.0006863664117418011


6×5 Array{Float64,2}:
 0.000686366  -9.31329e-5   -0.00182049  -0.00109524  -0.000998325
 0.000534584  -0.000478808  -0.00204866  -0.0020312   -0.0060328  
 0.000412001  -0.00102133   -0.00228859  -0.00272689  -0.00737199 
 0.000314161  -0.00177269   -0.00249474  -0.00305829  -0.00653947 
 0.000236978  -0.00282852   -0.00265241  -0.00307733  -0.00480155 
 0.000176801  -0.00433029   -0.00281343  -0.00297884  -0.00312101 

In [383]:
fun="debt"
debt_diffs = fetch(@spawn [TestFunctions.cvm_svm_comparison(svm2, c, p; fun=fun) 
                        for c in cgrid, p in pgrid])

println("Maximum ", fun, " difference: ", maximum(debt_diffs))
debt_diffs

Maximum debt difference: 0.0013777694208130242


6×5 Array{Float64,2}:
 0.00137777    0.00120022    0.00110941   -0.00458352    0.00131461 
 0.000859266   0.000656824   0.000798136  -0.00231916   -0.00264091 
 0.000328493   9.75211e-5    0.000234561  -0.000301232  -0.00378019 
 3.07125e-5   -0.000487657  -0.000307838  -0.000107049  -0.0032794  
 2.25028e-5   -0.00123696   -0.000828587  -0.000613446  -0.00209993 
 1.63085e-5   -0.00144357   -0.00148695   -0.00101569   -0.000961171

In [381]:
fun="equity"
eq_diffs = fetch(@spawn [TestFunctions.cvm_svm_comparison(svm2, c, p; fun=fun) 
                        for c in cgrid, p in pgrid])

println("Maximum ", fun, " difference: ", maximum(eq_diffs))
eq_diffs

Computing Equity Vmax
Vmax: 125.0
phi0: -42.51391951978036
phi1: 1.0
pv_rfdebt: 38.64330013215303
debt_abs_per_diff: 2.657196132952841e-5
eq_abs_per_diff: 0.0009688543375786891
Equity Vmax: 193.91602699731453
 
here
eq_max: 151.255420888986
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.010126113891601562
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 56.236411313504114
Equity Core Function Computation Time: 0.03574705123901367
Total Equity FD Core Function Computation Time: 0.12174105644226074
Total computation time: 0.14125514030456543
Computing Equity Vmax
Vmax: 125.0
phi0: -39.714305802868594
phi1: 1.0
pv_rfdebt: 39.12783580749282
debt_abs_per_diff: 8.039392698408944e-6
eq_abs_per_diff: 0.0008818181961766368
Equity Vmax: 203.61182834718028
 
here
eq_max: 163.75299472662385
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing 

equity: 35.9480609945465
Equity Core Function Computation Time: 0.035002946853637695
Total Equity FD Core Function Computation Time: 0.0797109603881836
Total computation time: 0.10840106010437012
Computing Equity Vmax
Vmax: 125.0
phi0: -56.70688076662812
phi1: 1.0
pv_rfdebt: 53.43737247879778
debt_abs_per_diff: 2.8775236726454635e-6
eq_abs_per_diff: 0.0009282489537589996
Equity Vmax: 272.8593235477421
 
here
eq_max: 215.95179950225
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.010135173797607422
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 38.177373875095526
Equity Core Function Computation Time: 0.03482508659362793
Total Equity FD Core Function Computation Time: 0.07950711250305176
Total computation time: 0.10817289352416992
Computing Equity Vmax
Vmax: 125.0
phi0: -53.90726704971644
phi1: 1.0
pv_rfdebt: 53.92190815413758
debt_abs_per_diff: 1.96793

equity: 14.80323209209515
Equity Core Function Computation Time: 0.0353550910949707
Total Equity FD Core Function Computation Time: 0.08015108108520508
Total computation time: 0.11540913581848145
Computing Equity Vmax
Vmax: 125.0
phi0: -70.89984201347596
phi1: 1.0
pv_rfdebt: 68.23144482544255
debt_abs_per_diff: 2.5515967789317238e-6
eq_abs_per_diff: 0.0009461670828778451
Equity Vmax: 348.2453238002054
 
here
eq_max: 277.08306662127796
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.010142087936401367
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 16.285564615354417
Equity Core Function Computation Time: 0.03531789779663086
Total Equity FD Core Function Computation Time: 0.08015799522399902
Total computation time: 0.11533904075622559
Computing Equity Vmax
Vmax: 125.0
phi0: -68.10022829656411
phi1: 1.0
pv_rfdebt: 68.71598050078234
debt_abs_per_diff: 1.83

6×5 Array{Float64,2}:
  0.000842203   0.000436365  -0.00111595  -0.00167012  -0.00053063
  0.000390727  -0.000221077  -0.00177619  -0.00247957  -0.00176175
 -0.00022809   -0.00104948   -0.00251132  -0.00328951  -0.00291229
 -0.00243879   -0.00233543   -0.00338548  -0.00413666  -0.00401612
 -0.00554371   -0.00547815   -0.00475363  -0.00509619  -0.0050684 
 -0.00830024   -0.00912407   -0.00715027  -0.00652935  -0.00618037

In [382]:
eq_diffs

6×5 Array{Float64,2}:
  0.000842203   0.000436365  -0.00111595  -0.00167012  -0.00053063
  0.000390727  -0.000221077  -0.00177619  -0.00247957  -0.00176175
 -0.00022809   -0.00104948   -0.00251132  -0.00328951  -0.00291229
 -0.00243879   -0.00233543   -0.00338548  -0.00413666  -0.00401612
 -0.00554371   -0.00547815   -0.00475363  -0.00509619  -0.0050684 
 -0.00830024   -0.00912407   -0.00715027  -0.00652935  -0.00618037

# Batch

In [1]:
using Distributions
using Interpolations
using Distributed
using DataFrames
using Printf
using JLD
using CSV
using Dierckx

main_path = "/home/artur/BondPricing"
module_path = string(main_path, "/", "Julia/modules/")
include(string(module_path, "/", "TestFunctions.jl"))
modls = ["Batch", "ModelObj", "AnalyticFunctions", 
         "BondPrInterp", "EqFinDiff"]
for modl in modls
    include(string(joinpath(module_path, modl), "/", modl, ".jl"))
end

include(string(module_path, "/", "TestFunctions.jl"))

ENV["LINES"] = 750

┌ Info: Recompiling stale cache file /home/artur/.julia/compiled/v1.1/AnalyticFunctions.ji for AnalyticFunctions [top-level]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/artur/.julia/compiled/v1.1/BondPrInterp.ji for BondPrInterp [top-level]
└ @ Base loading.jl:1184
┌ Info: Recompiling stale cache file /home/artur/.julia/compiled/v1.1/EqFinDiff.ji for EqFinDiff [top-level]
└ @ Base loading.jl:1184


750

## Batch - Part I

In [2]:
bt = Batch.BatchObj()
comb_num = 50 
bt = Batch.set_par_dict(bt, comb_num)

# Adjust parameters
# bt.mi._svm_dict["lambda"] = .3
# bt.mi._svm_dict["sigmal"] = .2
# bt.mi._svm_dict["sigmah"] = .2
# bt.mi._svm_dict["kappa"] = 0.015

# Set Directories & Paths (Main, Batch, Maturity, Combination)
bt = Batch.set_comb_res_paths(bt)

# Create Directories
bt = Batch.mk_comb_res_dirs(bt)

skip_c = true
eqfiles = [x for x in readdir(bt.mi.comb_res_path) if  occursin(bt.dfn.eq_fd_cp_fn_prefix, x)]
all_files = size(eqfiles, 1) == size(bt.coupon_grid, 1)
if all_files & skip_c
    println("All results have been computed. Skipping calculations...")
    c_counter = 999
end

# Construct Firm Object
svm = ModelObj.firm_constructor(bt.mi._svm_dict)

# Bond Pricing Interpolation - Inputs
svm.bi.vtmax = 1.2
svm.bi.vtN = 15
svm.bi.ttmN = 10
svm.bi.vbhlN = 11 

# Set Bond Pricing Variables' Grids
svm = ModelObj.set_bpr_grids(svm)

# Compute Bond Price Inputs
# svm = @time BondPrInterp.bpr_surfs(svm)

# Save Results
# Batch.save_batch_results(bt, svm=svm)

# Batch.check_batch_results(bt)
# # Load Bond Pricing Surfaces
res = Batch.load_batch_results(bt)

# # # # Interpolate Bond Pricing Surfaces
svm = @time BondPrInterp.bpr_interp(res["svm"])

All results have been computed. Skipping calculations...
Setting initial vbl value to NaN
Setting initial vbh value to NaN
Setting initial c value to NaN
Setting initial p value to NaN
 11.501913 seconds (14.23 M allocations: 810.207 MiB, 6.93% gc time)


Main.ModelObj.Firm
  mu_b: Float64 1.0
  m: Float64 1.0
  c: Float64 NaN
  p: Float64 NaN
  vbl: Float64 NaN
  vbh: Float64 NaN
  pm: Main.ModelObj.FirmParams
  bi: Main.ModelObj.BPrInputs
  bs: Main.ModelObj.BPrSurfs
  bf: Main.ModelObj.BPrInterpFuns


In [99]:
svm.c=4.5

# 1. Form P Grid 
# 2. Compute CVM VB value for sigmal and sigmah
# 3. Interpolate VB functions
# 4. Compute VB s.t. Debt is issued at par
# 5. Form Adjusted p Grid: [.85 x min(Opt P), 1.15 x max(Opt P)]
count = Batch.p_candidates(svm; c=svm.c)

# Bound VB Values: 
# For each p in Adjusted p-grid, find min(VB) and max(VB) s.t. .6 <= vbhl <= 1.4 
# that is, ratio of pre and post default barriers must be whitin bounds.
# Now, for each p in adjusted p-grid, I have an interval of VB candidate values
# for which debt is issued at par.
BVB = @time [vcat(p, [i for i in Batch.bounded_vbs(svm, count, p)]) for p in count["adj_pgrid"]]

# Store results in a DataFrame
df = DataFrame(transpose(hcat(BVB...)), [:p, :vblmin, :vblmax])

  0.288662 seconds (572.06 k allocations: 30.269 MiB, 5.94% gc time)


,p,vblmin,vblmax
,Float64,Float64,Float64
1,46.0869,35.1186,73.7442
2,46.9626,35.8263,75.2294
3,47.8382,36.5335,76.7146
4,48.7139,37.2404,78.1998
5,49.5896,37.9468,79.685
6,50.4652,38.6529,81.1702
7,51.3409,39.3641,82.6554
8,52.2165,40.0695,84.1406
9,53.0922,40.7745,85.6258


In [78]:
i = 1
Batch.svm_debt_at_par(svm, df[i, :vblmin], df[i, :vblmax], svm.c; p=df[i, :p])

vb root: 90.09738275297182
Condition 1: true
Condition 2: true
Condition 3: true
Conds: true
Counter: 0
Unique vb values: 16
Debt at par: preparing results...
returning results...
length: 16


,c,p,vb,debt,debt_diff,abs_debt_diff,debt_per_diff,abs_debt_per_diff,count,time
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,UInt64
1,7.0,58.3129,44.9438,59.3766,1.06376,1.06376,0.0182422,0.0182422,0,0x000000002585ba15
2,7.0,58.3129,48.8588,59.3768,1.06392,1.06392,0.0182451,0.0182451,0,0x000000002585ba15
3,7.0,58.3129,52.7738,59.3767,1.06382,1.06382,0.0182434,0.0182434,0,0x000000002585ba15
4,7.0,58.3129,56.6888,59.3766,1.06376,1.06376,0.0182424,0.0182424,0,0x000000002585ba15
5,7.0,58.3129,60.6038,59.3762,1.06333,1.06333,0.0182349,0.0182349,0,0x000000002585ba15
6,7.0,58.3129,64.5189,59.3734,1.0605,1.0605,0.0181863,0.0181863,0,0x000000002585ba15
7,7.0,58.3129,68.4339,59.3627,1.04982,1.04982,0.0180032,0.0180032,0,0x000000002585ba15
8,7.0,58.3129,72.3489,59.3302,1.01735,1.01735,0.0174464,0.0174464,0,0x000000002585ba15
9,7.0,58.3129,76.2639,59.2505,0.937622,0.937622,0.0160792,0.0160792,0,0x000000002585ba15


In [100]:
LL = @time fetch(@spawn [Batch.svm_debt_at_par(svm, df[i, :vblmin], df[i, :vblmax], svm.c; p=df[i, :p]) 
    for i in range(1, stop=size(df, 1))])

DFI = vcat(LL...)

vb root: 73.07296065372587
Condition 1: true
Condition 2: true
Condition 3: true
Conds: true
Counter: 0
Unique vb values: 16
Debt at par: preparing results...
returning results...
length: 16
vb root: 74.51739271495771
Condition 1: true
Condition 2: true
Condition 3: true
Conds: true
Counter: 0
Unique vb values: 16
Debt at par: preparing results...
returning results...
length: 16
vb root: 75.91542393664444
Condition 1: true
Condition 2: true
Condition 3: true
Conds: true
Counter: 0
Unique vb values: 16
Debt at par: preparing results...
returning results...
length: 16
vb root: 77.28177189014909
Condition 1: true
Condition 2: true
Condition 3: true
Conds: true
Counter: 0
Unique vb values: 16
Debt at par: preparing results...
returning results...
length: 16
vb root: 78.65301318083705
Condition 1: true
Condition 2: true
Condition 3: true
Conds: true
Counter: 0
Unique vb values: 16
Debt at par: preparing results...
returning results...
length: 16
vb root: 79.9970034942888
Condition 1: true
C

,c,p,vb,debt,debt_diff,abs_debt_diff,debt_per_diff,abs_debt_per_diff,count,time
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,UInt64
1,4.5,46.0869,36.0164,46.4265,0.339547,0.339547,0.00736754,0.00736754,0,0x000000001ac46531
2,4.5,46.0869,39.1538,46.4265,0.339547,0.339547,0.00736754,0.00736754,0,0x000000001ac46531
3,4.5,46.0869,42.2912,46.4265,0.339547,0.339547,0.00736754,0.00736754,0,0x000000001ac46531
4,4.5,46.0869,45.4285,46.4263,0.339338,0.339338,0.007363,0.007363,0,0x000000001ac46531
5,4.5,46.0869,48.5659,46.4263,0.339339,0.339339,0.00736301,0.00736301,0,0x000000001ac46531
6,4.5,46.0869,51.7032,46.4263,0.33934,0.33934,0.00736305,0.00736305,0,0x000000001ac46531
7,4.5,46.0869,54.8406,46.4263,0.339326,0.339326,0.00736274,0.00736274,0,0x000000001ac46531
8,4.5,46.0869,57.978,46.4262,0.339255,0.339255,0.00736119,0.00736119,0,0x000000001ac46531
9,4.5,46.0869,61.1153,46.4259,0.338948,0.338948,0.00735453,0.00735453,0,0x000000001ac46531


In [164]:
# df2 = df
i = 2
c = svm.c 
N1=15
N2=10^5
f_low=.85
f_high=1.15
debt_vars=bt.dfc.debt_vars 
disp_option=true

vbl=NaN
p=df2[i, :p]
x_min= df2[i, :vblmin] 
x_max= df2[i, :vblmax]

# Initial Check:
if isnan(p) & isnan(vbl)
    println("Please enter a value for p or vbl")
    return
end

# Form Grids:
xvar = [:p, :vb][isnan.([p, vbl])][1]
fixed_var = [:p, :vb][isnan.([p, vbl]).==false][1]
fixed_var_value = [p, vbl][isnan.([p, vbl]).==false][1]
xmin, xmax = Batch.debt_at_par_grid_bounds(svm, c, x_min, x_max; 
                                     p=p, vbl=vbl, f_low=.75, f_high=1.25)
xgrid = range(xmin, stop=xmax, length=N1)
xgrid_ref = range(xmin, stop=xmax, length=N2)

# Compute Debt Values
df = DataFrame(hcat(reverse(xgrid), 999. * ones(length(xgrid), 5)),
                    vcat([xvar, :debt], debt_vars))

if xvar == :p
    df[:debt] = fetch(@spawn [get_svm_debt_price(svm, vbl; c=c, p=pval) for pval in df[:p]])
else
    df[:debt] = fetch(@spawn [BondPrInterp.get_svm_debt_price(svm, vblval; c=c, p=p) for vblval in df[:vb]])
end

# Sort DataFrame
sort!(df, xvar)

# Compute Candidate Solution:
df = Batch.debt_at_par_main(svm, c, df, xvar, xgrid_ref,
                      vbl=vbl, p=p, disp_option=disp_option)



0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.02695742762795591
returning CVM bond price
0.02695742762795591
returning CVM bond price
0.02695742762795591
returning CVM bond price
0.02695742762795591
returning CVM bond price
0.02695742762795591
returning CVM bond price
0.02695742762795591
returning CVM bond price
0.02695742762795591
returning CVM bond price
0.02695742762795591
returning CVM bond price
0.02695742762795591
returning CVM bond price
0.02695742762795591
returning CVM bond price
0.02695742762795591
returning CVM bond price
0.02695742762795591
returning CVM bond price
0.02695742762795591
returning CVM bond price
0.0269574276279

0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504
0.08315550177498504


0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518994267
0.3844394518

MethodError: MethodError: no method matching Spline1D(::StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}, ::RemoteException; k=3, bc="extrapolate")
Closest candidates are:
  Spline1D(::Any, ::Any, !Matched::Any, !Matched::Any, !Matched::Any) at /home/artur/.julia/packages/Dierckx/9SY4A/src/Dierckx.jl:88 got unsupported keyword arguments "k", "bc"
  Spline1D(::Any, ::Any, !Matched::Any, !Matched::Any, !Matched::Any, !Matched::Any) at /home/artur/.julia/packages/Dierckx/9SY4A/src/Dierckx.jl:79 got unsupported keyword arguments "k", "bc"
  Spline1D(::AbstractArray{T,1} where T, !Matched::AbstractArray{T,1} where T; w, k, s, bc, periodic) at /home/artur/.julia/packages/Dierckx/9SY4A/src/Dierckx.jl:124
  ...

In [ ]:
0.4657924609205794

In [ ]:
for modl in modls
    include(string(joinpath(module_path, modl), "/", modl, ".jl"))
end

In [165]:
BondPrInterp.get_svm_debt_price(svm, 98.51761018242067; c=5.0, p=9.968120496260154) 

0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM bond price
0.014934870212052672
returning CVM

70.10232195801863

In [134]:
fetch(@spawn [get_svm_debt_price(svm, vblval; c=c, p=p) for vblval in df[:vb]])

RemoteException(1, CapturedException(UndefVarError(:get_svm_debt_price), Any[((::##162#164)(::Float64) at none:0, 1), (iterate at generator.jl:47 [inlined], 1), (collect(::Generator{Array{Float64,1},##162#164}) at array.jl:606, 1), ((::##161#163)() at macros.jl:43, 1), (run_work_thunk(::##161#163, ::Bool) at process_messages.jl:56, 1), (run_work_thunk(::RemoteValue, ::Function) at process_messages.jl:65, 1), ((::##102#104{RemoteValue,##161#163})() at task.jl:259, 1)]))

In [118]:
i = 2
Batch.svm_debt_at_par(svm, df[i, :vblmin], df[i, :vblmax], svm.c; p=df[i, :p]) 

MethodError: MethodError: Cannot `convert` an object of type RemoteException to an object of type Float64
Closest candidates are:
  convert(::Type{T<:AbstractFloat}, !Matched::Ratios.SimpleRatio{S}) where {T<:AbstractFloat, S} at /home/artur/.julia/packages/Ratios/ZrTTx/src/Ratios.jl:16
  convert(::Type{T<:Number}, !Matched::T<:Number) where T<:Number at number.jl:6
  convert(::Type{T<:Number}, !Matched::Number) where T<:Number at number.jl:7
  ...

In [101]:
debt_at_par_cp_fname = string(bt.dfn.debt_at_par_cp_fn_prefix,
                                  @sprintf("%.2f", svm.c), ".csv")
debt_at_par_cp_path_fname = joinpath(bt.mi.comb_res_path,  debt_at_par_cp_fname)
CSV.write(debt_at_par_cp_path_fname, DFI)

"/mnt/ide0/home/artur/BondPricing/Julia/Results/m_1.0/xi_1.00__kappa_25_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20/debt_at_par_c_4.50.csv"

In [284]:
readdir(bt.mi.comb_res_path)

3-element Array{String,1}:
 "batch_obj.jld"         
 "bond_surfaces.jld"     
 "debt_at_par_c_5.00.csv"

In [388]:
using CSV

In [285]:
CSV.read(debt_at_par_cp_path_fname)

,c,p,vb,debt,debt_diff,abs_debt_diff,debt_per_diff,abs_debt_per_diff,count,time
,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Int64⍰,Int64⍰
1,5.0,25.4358,48.9556,88.0555,11.748,11.748,0.153956,0.153956,1,1135860948
2,5.0,25.4358,50.4647,88.0685,11.761,11.761,0.154127,0.154127,1,1135860948
3,5.0,25.4358,51.9739,88.0722,11.7647,11.7647,0.154175,0.154175,1,1135860948
4,5.0,25.4358,53.483,88.0715,11.764,11.764,0.154166,0.154166,1,1135860948
5,5.0,25.4358,54.9922,88.062,11.7545,11.7545,0.154042,0.154042,1,1135860948
6,5.0,25.4358,56.5013,88.0457,11.7382,11.7382,0.153828,0.153828,1,1135860948
7,5.0,25.4358,58.0105,88.0253,11.7178,11.7178,0.15356,0.15356,1,1135860948
8,5.0,25.4358,59.5197,87.9878,11.6803,11.6803,0.153069,0.153069,1,1135860948
9,5.0,25.4358,61.0288,87.943,11.6355,11.6355,0.152482,0.152482,1,1135860948


In [389]:
DFI2 = CSV.read(debt_at_par_cp_path_fname; types=eltypes(DFI))

MethodError: MethodError: no method matching eltypes(::Array{Any,1})
Closest candidates are:
  eltypes(!Matched::AbstractDataFrame) at /home/artur/.julia/packages/DataFrames/IKMvt/src/abstractdataframe/abstractdataframe.jl:207

## Batch Part II

In [62]:
using Distributions
using Interpolations
using Distributed
using DataFrames
using Printf
using JLD
using CSV
using Dierckx

main_path = "/home/artur/BondPricing"
module_path = string(main_path, "/", "Julia/modules/")
include(string(module_path, "/", "TestFunctions.jl"))
modls = ["Batch", "ModelObj", "AnalyticFunctions", 
         "BondPrInterp", "EqFinDiff"]
for modl in modls
    include(string(joinpath(module_path, modl), "/", modl, ".jl"))
end

include(string(module_path, "/", "TestFunctions.jl"))

ENV["LINES"] = 750

750

In [96]:
svm_c_step = .5
svm_coupon_grid = range(svm_c_step, stop=11 + svm_c_step, step=svm_c_step)



0.5:0.5:11.5

In [4]:
svm.c=10.5
debt_at_par_cp_fname = string(bt.dfn.debt_at_par_cp_fn_prefix,
                                  @sprintf("%.2f", svm.c), ".csv")
debt_at_par_cp_path_fname = joinpath(bt.mi.comb_res_path,  debt_at_par_cp_fname)
DFI = Batch.conditionally_load_df(:DFI2, debt_at_par_cp_path_fname)

,c,p,vb,debt,debt_diff,abs_debt_diff,debt_per_diff,abs_debt_per_diff,count,time
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,UInt64
1,10.5,61.6439,46.8983,64.149,2.50513,2.50513,0.0406387,0.0406387,1,0x0000000004541a83
2,10.5,61.6439,48.7948,64.1558,2.51184,2.51184,0.0407476,0.0407476,1,0x0000000004541a83
3,10.5,61.6439,50.6912,64.1491,2.50516,2.50516,0.0406392,0.0406392,1,0x0000000004541a83
4,10.5,61.6439,52.5877,64.158,2.51405,2.51405,0.0407835,0.0407835,1,0x0000000004541a83
5,10.5,61.6439,54.4842,64.1455,2.50154,2.50154,0.0405806,0.0405806,1,0x0000000004541a83
6,10.5,61.6439,56.3807,64.1593,2.51539,2.51539,0.0408052,0.0408052,1,0x0000000004541a83
7,10.5,61.6439,58.2772,64.1481,2.50414,2.50414,0.0406226,0.0406226,1,0x0000000004541a83
8,10.5,61.6439,60.1737,64.1492,2.50532,2.50532,0.0406418,0.0406418,1,0x0000000004541a83
9,10.5,61.6439,62.0702,64.156,2.5121,2.5121,0.0407518,0.0407518,1,0x0000000004541a83


In [5]:
# Load DFI 
df = DFI #Batch.conditionally_load_df(:DFI, debt_at_par_cp_path_fname)

# Filter DataFrame
fdf = Batch.filter_batch_I_df(bt, svm, df; N1=10^5, tol=.05)

# DataFrame Size:
println(string("Part I DF length: ", size(df, 1)))
println(string("Part I Filtered DF length: ", size(fdf, 1)))

# Apply Finite Differences Method
# eqdf_all = Batch.eq_fd_method(bt, svm, fdf)
fdf

Part I DF length: 458
Part I Filtered DF length: 352


,c,p,vb,debt,debt_diff,abs_debt_diff,debt_per_diff,abs_debt_per_diff,count,time
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,UInt64
1,10.5,61.6439,46.8983,64.149,2.50513,2.50513,0.0406387,0.0406387,1,0x0000000004541a83
2,10.5,61.6439,48.7948,64.1558,2.51184,2.51184,0.0407476,0.0407476,1,0x0000000004541a83
3,10.5,61.6439,50.6912,64.1491,2.50516,2.50516,0.0406392,0.0406392,1,0x0000000004541a83
4,10.5,61.6439,52.5877,64.158,2.51405,2.51405,0.0407835,0.0407835,1,0x0000000004541a83
5,10.5,61.6439,54.4842,64.1455,2.50154,2.50154,0.0405806,0.0405806,1,0x0000000004541a83
6,10.5,61.6439,56.3807,64.1593,2.51539,2.51539,0.0408052,0.0408052,1,0x0000000004541a83
7,10.5,61.6439,58.2772,64.1481,2.50414,2.50414,0.0406226,0.0406226,1,0x0000000004541a83
8,10.5,61.6439,60.1737,64.1492,2.50532,2.50532,0.0406418,0.0406418,1,0x0000000004541a83
9,10.5,61.6439,62.0702,64.156,2.5121,2.5121,0.0407518,0.0407518,1,0x0000000004541a83


In [6]:
eqdf_all = Batch.eq_fd_method(bt, svm, fdf)

Computing Equity Vmax
Vmax: 125.0
phi0: -32.77615630389795
phi1: 1.0
pv_rfdebt: 64.20108456507828
debt_abs_per_diff: 3.927426327742956e-6
eq_abs_per_diff: 0.0009143756504647899
Equity Vmax: 444.4590859930447
 
eq_max: 411.68209541686883
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.08588814735412598
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 95.60172304859654
Equity Core Function Computation Time: 1.8959360122680664
Total Equity FD Core Function Computation Time: 3.4088659286499023
Total computation time: 6.95339298248291
Computing Equity Vmax
Vmax: 125.0
phi0: -32.77615630389795
phi1: 1.0
pv_rfdebt: 64.20108456507828
debt_abs_per_diff: 3.927426327742956e-6
eq_abs_per_diff: 0.0009143756504647899
Equity Vmax: 444.4590859930447
 
eq_max: 411.68209541686866
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatilit

debt_abs_per_diff: 3.927426327742956e-6
eq_abs_per_diff: 0.0009143756504647899
Equity Vmax: 444.4590859930447
 
eq_max: 411.68209541686883
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.013881921768188477
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 40.79379310943412
Equity Core Function Computation Time: 0.08099794387817383
Total Equity FD Core Function Computation Time: 0.153839111328125
Total computation time: 0.22738099098205566
Computing Equity Vmax
Vmax: 125.0
phi0: -32.77615630389795
phi1: 1.0
pv_rfdebt: 64.20108456507828
debt_abs_per_diff: 3.927426327742956e-6
eq_abs_per_diff: 0.0009143756504647899
Equity Vmax: 444.4590859930447
 
eq_max: 411.68209541686883
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.01378488540649414
 
Shape

equity: 7.372302418962084
Equity Core Function Computation Time: 0.1035909652709961
Total Equity FD Core Function Computation Time: 0.17775392532348633
Total computation time: 0.2551918029785156
Computing Equity Vmax
Vmax: 125.0
phi0: -32.77615630389795
phi1: 1.0
pv_rfdebt: 64.20108456507828
debt_abs_per_diff: 3.927426327742956e-6
eq_abs_per_diff: 0.0009143756504647899
Equity Vmax: 444.4590859930447
 
eq_max: 411.68209541686883
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.014161109924316406
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 7.238515432545087
Equity Core Function Computation Time: 0.042758941650390625
Total Equity FD Core Function Computation Time: 0.28506898880004883
Total computation time: 0.35950589179992676
Computing Equity Vmax
Vmax: 125.0
phi0: -32.77615630389795
phi1: 1.0
pv_rfdebt: 64.20108456507828
debt_abs_per_diff: 3.927426327

Computing equity values... 
equity: 67.70484920998274
Equity Core Function Computation Time: 0.04468202590942383
Total Equity FD Core Function Computation Time: 0.2881770133972168
Total computation time: 0.36156797409057617
Computing Equity Vmax
Vmax: 125.0
phi0: -34.450385698195554
phi1: 1.0
pv_rfdebt: 65.77682982556321
debt_abs_per_diff: 3.7315285907965912e-6
eq_abs_per_diff: 0.0009998950437468802
Equity Vmax: 444.4590859930447
 
eq_max: 410.00770023095254
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.013873100280761719
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 63.25503784153549
Equity Core Function Computation Time: 0.08022403717041016
Total Equity FD Core Function Computation Time: 0.1541750431060791
Total computation time: 0.22800302505493164
Computing Equity Vmax
Vmax: 125.0
phi0: -34.450385698195554
phi1: 1.0
pv_rfdebt: 65.77682982556321


debt_abs_per_diff: 3.7315285907965912e-6
eq_abs_per_diff: 0.0009998950437468802
Equity Vmax: 444.4590859930447
 
eq_max: 410.00770023095254
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.013933181762695312
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 20.989848117868597
Equity Core Function Computation Time: 0.08360910415649414
Total Equity FD Core Function Computation Time: 0.1557469367980957
Total computation time: 0.2288520336151123
Computing Equity Vmax
Vmax: 125.0
phi0: -34.450385698195554
phi1: 1.0
pv_rfdebt: 65.77682982556321
debt_abs_per_diff: 3.7315285907965912e-6
eq_abs_per_diff: 0.0009998950437468802
Equity Vmax: 444.4590859930447
 
eq_max: 410.00770023095254
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.013937950134277344
 


...

debt_abs_per_diff: 2.0669242940213705e-6
eq_abs_per_diff: 0.0009578641101947319
Equity Vmax: 567.254936738525
 
eq_max: 514.3866019316241
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.013983011245727539
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 11.07145811437102
Equity Core Function Computation Time: 0.07949280738830566
Total Equity FD Core Function Computation Time: 0.1515209674835205
Total computation time: 0.23320293426513672
Computing Equity Vmax
Vmax: 125.0
phi0: -52.866909035468204
phi1: 1.0
pv_rfdebt: 83.1100276908975
debt_abs_per_diff: 2.0669242940213705e-6
eq_abs_per_diff: 0.0009578641101947319
Equity Vmax: 567.254936738525
 
eq_max: 514.386601931624
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.01599287986755371
 
Shape o

Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.010577201843261719
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 22.48488457055801
Equity Core Function Computation Time: 0.06894397735595703
Total Equity FD Core Function Computation Time: 0.1309499740600586
Total computation time: 0.1867060661315918
Computing Equity Vmax
Vmax: 125.0
phi0: -62.91228540125316
phi1: 1.0
pv_rfdebt: 92.56449925380709
debt_abs_per_diff: 1.4217187877345448e-6
eq_abs_per_diff: 0.0009874698701695722
Equity Vmax: 625.3985677542238
 
eq_max: 562.4844964954917
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.010322809219360352
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 17.71195872319585
Equity Core Function Computation Time: 0.039697885513305664
Total Equity FD Core Function Computation Time: 0.2195

equity: 35.29096307635118
Equity Core Function Computation Time: 0.07050704956054688
Total Equity FD Core Function Computation Time: 0.1314530372619629
Total computation time: 0.18852901458740234
Computing Equity Vmax
Vmax: 125.0
phi0: -64.58651479555068
phi1: 1.0
pv_rfdebt: 94.14024451429202
debt_abs_per_diff: 1.3267841012212338e-6
eq_abs_per_diff: 0.0008763284538308359
Equity Vmax: 656.668496141935
 
eq_max: 592.0804472648167
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.010311126708984375
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 30.64377510092239
Equity Core Function Computation Time: 0.04119992256164551
Total Equity FD Core Function Computation Time: 0.21502900123596191
Total computation time: 0.26811909675598145
Computing Equity Vmax
Vmax: 125.0
phi0: -64.58651479555068
phi1: 1.0
pv_rfdebt: 94.14024451429202
debt_abs_per_diff: 1.3267841012

Computing equity values... 
equity: 0.09055678897612487
Equity Core Function Computation Time: 0.04119610786437988
Total Equity FD Core Function Computation Time: 0.2129230499267578
Total computation time: 0.26612210273742676
108.227631 seconds (42.67 M allocations: 49.882 GiB, 34.32% gc time)


,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,m,c,p,vb,debt_diff,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,abs_debt_diff,debt_per_diff,abs_debt_per_diff,debt,equity,firm_value,leverage,ROE,V0,r,alpha,pi,xi
,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float32,Float32,Float32,Float32,Float32
1,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,61.6439,46.8983,2.50513,0.959926,-1.02053,0.0711092,false,0.0,2.50513,0.0406387,0.0406387,64.149,95.6017,159.751,40.1557,166.664,100.0,0.08,0.6,0.27,1.0
2,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,61.6439,48.7948,2.51184,0.95089,-0.726086,0.071721,false,0.0,2.51184,0.0407476,0.0407476,64.1558,89.7111,153.867,41.6956,150.28,100.0,0.08,0.6,0.27,1.0
3,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,61.6439,50.6912,2.50516,0.943739,-0.453629,0.0724102,false,0.0,2.50516,0.0406392,0.0406392,64.1491,84.1614,148.311,43.2532,134.754,100.0,0.08,0.6,0.27,1.0
4,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,61.6439,52.5877,2.51405,0.920517,-0.200779,0.0718456,false,0.0,2.51405,0.0407835,0.0407835,64.158,78.9121,143.07,44.8437,120.166,100.0,0.08,0.6,0.27,1.0
5,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,61.6439,54.4842,2.50154,0.882482,0.0345167,0.0700566,false,0.0,2.50154,0.0405806,0.0405806,64.1455,73.9257,138.071,46.4583,106.182,100.0,0.08,0.6,0.27,1.0
6,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,61.6439,56.3807,2.51539,0.213758,0.213758,0.0193168,false,0.0,2.51539,0.0408052,0.0408052,64.1593,69.1606,133.32,48.1243,92.9666,100.0,0.08,0.6,0.27,1.0
7,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,61.6439,58.2772,2.50414,-0.359423,-0.359423,-0.0784037,true,0.0,2.50414,0.0406226,0.0406226,64.1481,64.6005,128.749,49.8243,80.1868,100.0,0.08,0.6,0.27,1.0
8,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,61.6439,60.1737,2.50532,-0.732073,-0.732073,-0.356083,true,0.0,2.50532,0.0406418,0.0406418,64.1492,60.2313,124.381,51.575,68.0057,100.0,0.08,0.6,0.27,1.0
9,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,61.6439,62.0702,2.5121,-0.871238,-0.871238,-0.540461,true,0.0,2.5121,0.0407518,0.0407518,64.156,56.0379,120.194,53.3771,56.3384,100.0,0.08,0.6,0.27,1.0


In [67]:
eqdf_all[abs.(eqdf_all[:p] .- unique(eqdf_all[:p])[3]).<1e-4, :]

,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,m,c,p,vb,debt_diff,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,abs_debt_diff,debt_per_diff,abs_debt_per_diff,debt,equity,firm_value,leverage,ROE,V0,r,alpha,pi,xi
,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float32,Float32,Float32,Float32,Float32
1,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,4.0,49.7557,39.0585,-0.0147072,-2.26499,-2.26499,-1.56911,true,0.0,0.0147072,-0.000295588,0.000295588,49.741,84.6016,134.343,37.0255,68.3312,100.0,0.08,0.6,0.27,1.0
2,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,4.0,49.7557,42.4608,-0.0147072,-1.65108,-1.65108,-1.06187,true,0.0,0.0147072,-0.000295588,0.000295588,49.741,77.6674,127.408,39.0406,54.5343,100.0,0.08,0.6,0.27,1.0
3,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,4.0,49.7557,45.8631,-0.0148849,-1.11094,-1.11094,-0.608426,true,0.0,0.0148849,-0.000299159,0.000299159,49.7408,71.6691,121.41,40.9693,42.599,100.0,0.08,0.6,0.27,1.0
4,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,4.0,49.7557,49.2654,-0.0148886,-0.619897,-0.619897,-0.238623,true,0.0,0.0148886,-0.000299234,0.000299234,49.7408,66.3946,116.135,42.83,32.1044,100.0,0.08,0.6,0.27,1.0
5,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,4.0,49.7557,52.6677,-0.014896,-0.152225,-0.152225,-0.0181171,false,0.0,0.014896,-0.000299383,0.000299383,49.7408,61.6841,111.425,44.6406,22.732,100.0,0.08,0.6,0.27,1.0
6,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,4.0,49.7557,56.07,-0.0149315,0.280066,0.280066,0.0168581,false,0.0,0.0149315,-0.000300096,0.000300096,49.7408,57.4112,107.152,46.4207,14.2302,100.0,0.08,0.6,0.27,1.0
7,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,4.0,49.7557,59.4723,-0.0151362,0.659707,0.659707,0.0391983,false,0.0,0.0151362,-0.000304211,0.000304211,49.7406,53.4745,103.215,48.1912,6.3969,100.0,0.08,0.6,0.27,1.0
8,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,4.0,49.7557,62.8746,-0.0159384,0.992195,0.992195,0.0594898,false,0.0,0.0159384,-0.000320333,0.000320333,49.7398,49.7929,99.5326,49.9733,-0.929927,100.0,0.08,0.6,0.27,1.0
9,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,4.0,49.7557,66.2769,-0.018498,1.29369,1.03719,0.0784405,false,0.0,0.018498,-0.000371776,0.000371776,49.7372,46.3022,96.0394,51.7883,-7.87986,100.0,0.08,0.6,0.27,1.0


In [7]:
eqdf_final = Batch.eq_fd_processing(bt, svm, eqdf_all)

,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,m,c,p,vb,debt_diff,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,abs_debt_diff,debt_per_diff,abs_debt_per_diff,debt,equity,firm_value,leverage,ROE,V0,r,alpha,pi,xi
,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float32,Float32,Float32,Float32,Float32
1,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,61.6439,73.1127,2.47048,6.43054e-6,7.20332e-6,0.00173401,false,0.0,2.47048,0.0400766,0.0400766,64.1144,34.5216,98.636,65.001,-3.80091,100.0,0.08,0.6,0.27,1.0
2,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,63.2876,75.1993,2.34421,8.30051e-6,8.06456e-6,0.00228359,false,0.0,2.34421,0.0370406,0.0370406,65.6318,31.2851,96.9169,67.7197,-8.97082,100.0,0.08,0.6,0.27,1.0
3,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,64.9312,77.2841,2.16244,-5.02912e-6,-5.05398e-6,0.00232919,false,0.0,2.16244,0.0333036,0.0333036,67.0937,28.0059,95.0996,70.551,-14.892,100.0,0.08,0.6,0.27,1.0
4,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,66.5749,79.3699,1.89862,3.43245e-6,3.45476e-6,0.00180541,false,0.0,1.89862,0.0285185,0.0285185,68.4735,24.6939,93.1674,73.4952,-21.6726,100.0,0.08,0.6,0.27,1.0
5,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,68.2186,81.4548,1.52483,-1.78561e-6,-1.79766e-6,0.00165517,false,0.0,1.52483,0.0223521,0.0223521,69.7434,21.3716,91.115,76.5444,-29.3656,100.0,0.08,0.6,0.27,1.0
6,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,69.8622,83.5411,0.989101,1.16308e-5,1.16327e-5,0.00157771,false,0.0,0.989101,0.0141579,0.0141579,70.8513,18.0647,88.916,79.6834,-38.0258,100.0,0.08,0.6,0.27,1.0
7,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,71.5059,65.3266,1.71252,3.30812e-6,0.0186223,0.00529579,false,0.0,1.71252,0.0239494,0.0239494,73.2184,53.7927,127.011,57.6473,100.857,100.0,0.08,0.6,0.27,1.0
8,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,73.1496,87.7118,-0.676128,-3.46725e-7,-7.3612e-8,0.00169317,false,0.0,0.676128,-0.00924309,0.00924309,72.4734,11.6569,84.1303,86.1443,-57.6523,100.0,0.08,0.6,0.27,1.0
9,0.02,0.02,0.0,0.005,0.75,0.225,0.15,1.0,1.0,10.5,74.7932,89.7942,-2.08968,9.46723e-6,9.36232e-6,0.00313913,false,0.0,2.08968,-0.0279394,0.0279394,72.7035,8.66065,81.3642,89.3557,-68.2719,100.0,0.08,0.6,0.27,1.0


In [106]:
res = Batch.eq_fd_sol(bt, svm, eqdf_final)

,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,m,c,p,vb,debt_diff,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,abs_debt_diff,debt_per_diff,abs_debt_per_diff,debt,equity,firm_value,leverage,ROE,V0,r,alpha,pi,xi
,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float32,Float32,Float32,Float32,Float32
1,0.02,0.02,0.0,0.0025,0.1,0.2,0.15,1.0,1.0,4.5,54.5244,59.3214,1.34048e-6,1.44767e-5,0.00110436,0.00771135,false,-1.20908e-6,1.34048e-6,2.61588e-8,2.61588e-8,54.5244,54.6095,109.134,49.961,20.0854,100.0,0.08,0.6,0.27,1.0


In [108]:
bt.mi.comb_res_path

"/mnt/ide0/home/artur/BondPricing/Julia/Results/m_1.0/xi_1.00__kappa_25_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20"

In [14]:
# sol = sort!(vcat(eqdf_final[bt.dfc.dfcols], res[bt.dfc.dfcols]), [:p])

# # Identify solution
# sol[:opt] = (abs.(sol[:abs_debt_diff] .- 
#                      minimum(sol[:abs_debt_diff])) .< 1e-4) .* 
#                     (abs.(sol[:eq_deriv]) .< 1e-4)



In [107]:
eq_fd_cp_fname = string(bt.dfn.eq_fd_cp_fn_prefix, @sprintf("%.2f", svm.c), ".csv")
eq_fd_cp_path_fname = joinpath(bt.mi.comb_res_path,  eq_fd_cp_fname)

Batch.save_eq_results(bt, res, eqdf_final, eq_fd_cp_path_fname)

"/mnt/ide0/home/artur/BondPricing/Julia/Results/m_1.0/xi_1.00__kappa_25_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20/eq_fd_c_4.50.csv"

In [17]:
col_types = vcat(fill(Float64, 16), [Bool], fill(Float64, 14), [Bool])
sol2 = CSV.read(eq_fd_cp_path_fname, types=col_types)

,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,m,c,p,vb,debt_diff,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,abs_debt_diff,debt_per_diff,abs_debt_per_diff,debt,equity,firm_value,leverage,ROE,V0,r,alpha,pi,xi,opt
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool
1,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,1.0,9.4453,10.457,0.0497651,1.39227e-5,1.85147e-5,0.000537695,false,-7.83279e-8,0.0497651,0.00526877,0.00526877,9.49506,92.6854,102.18,9.29245,2.40921,100.0,0.08,0.6,0.27,1.0,false
2,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,1.0,9.61149,10.6529,0.0421143,9.15168e-6,1.40656e-5,0.000549222,false,-7.69028e-8,0.0421143,0.00438166,0.00438166,9.65361,92.4909,102.144,9.45093,2.37361,100.0,0.08,0.6,0.27,1.0,false
3,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,1.0,9.77768,10.8488,0.0344636,-5.08565e-6,-5.65182e-6,0.000362255,false,-3.06777e-8,0.0344636,0.00352472,0.00352472,9.81215,92.2957,102.108,9.6096,2.33714,100.0,0.08,0.6,0.27,1.0,false
4,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,1.0,9.94388,11.0446,0.0268128,2.37225e-6,9.85252e-7,0.000575842,false,-5.40353e-8,0.0268128,0.00269641,0.00269641,9.97069,92.1024,102.073,9.76818,2.30273,100.0,0.08,0.6,0.27,1.0,false
5,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,1.0,10.1101,11.2404,0.019162,-2.25513e-6,4.20571e-6,0.000591643,false,6.05799e-7,0.019162,0.00189534,0.00189534,10.1292,91.9085,102.038,9.92694,2.26745,100.0,0.08,0.6,0.27,1.0,false
6,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,1.0,10.2763,11.4363,0.0115113,2.98256e-6,2.11471e-6,0.000606889,false,-1.00375e-8,0.0115113,0.00112018,0.00112018,10.2878,91.7146,102.002,10.0858,2.23203,100.0,0.08,0.6,0.27,1.0,false
7,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,1.0,10.4425,11.6321,0.00386052,-1.52825e-5,-8.03487e-6,0.000623075,false,7.29203e-7,0.00386052,0.000369695,0.000369695,10.4463,91.5214,101.968,10.2447,2.1972,100.0,0.08,0.6,0.27,1.0,false
8,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,1.0,10.5263,11.7309,-1.66729e-7,-2.42744e-6,7.12015e-6,0.000637466,false,6.46128e-7,1.66729e-7,-1.60162e-8,1.60162e-8,10.5263,91.4235,101.95,10.325,2.17921,100.0,0.08,0.6,0.27,1.0,true
9,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,1.0,10.6086,11.828,-0.00379024,9.26516e-6,1.66497e-5,0.000638632,false,3.24414e-7,0.00379024,-0.000357278,0.000357278,10.6049,91.3274,101.932,10.4038,2.16149,100.0,0.08,0.6,0.27,1.0,false


# Collect Results

In [443]:
using Distributions
using Interpolations
using Distributed
using DataFrames
using Printf
using JLD
using CSV
using Dierckx

main_path = "/home/artur/BondPricing"
module_path = string(main_path, "/", "Julia/modules/")
include(string(module_path, "/", "TestFunctions.jl"))
modls = ["Batch", "ModelObj", "AnalyticFunctions", 
         "BondPrInterp", "EqFinDiff"]
for modl in modls
    include(string(joinpath(module_path, modl), "/", modl, ".jl"))
end

include(string(module_path, "/", "TestFunctions.jl"))

ENV["LINES"] = 750

750

In [444]:
bt = Batch.BatchObj()

Main.Batch.BatchStruct
  dfn: Main.Batch.DirFileNames
  dfc: Main.Batch.BatchDFColumns
  mi: Main.Batch.ModelInputs
  svm_param_values_dict: Dict{String,Array{T,1} where T}
  svm_coupon_grid: StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}
  pvb_tol_vec: Array{Float64}((4,)) [1.0e-6, 1.0e-5, 0.0001, 0.001]
  _common_params: Dict{String,Float64}
  _params_order: Array{String}((9,))
  _params_combs: Array{Array{Float64,1}}((1, 4, 1, 5, 1, 5, 1, 1, 6))
  coupon_grid: StepRange{Int64,Int64}


In [322]:
Batch.diag_df(bt, 121)

,V0,alpha,gross_delta,iota,kappa,lambda,m,mu_b,pi,r,sigmah,sigmal,xi,comb_num,folder_name,folder_created,count
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,String,Bool,Int64
1,100.0,0.6,0.02,0.0,0.005,0.75,1.0,1.0,0.27,0.08,0.2,0.15,1.0,121,xi_1.00__kappa_50_bp__gross_delta_200_bp__iota_0_bp__lambda_0.75__sigmal_0.15__sigmah_0.20,true,11


In [445]:
ENV["LINES"] = 750
diagdf = Batch.diagnosis(bt)
sort(diagdf, [:m, :comb_num])

,comb_num,m,folder_created,batch_obj,bond_surfs,count,folder_name,V0,alpha,gross_delta,iota,kappa,lambda,mu_b,pi,r,sigmah,sigmal,xi
,Int64,Float64,Bool,Bool,Bool,Int64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,3.0,true,true,true,11,xi_1.00__kappa_10_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.001,0.1,1.0,0.27,0.08,0.2,0.15,1.0
2,5,3.0,true,true,true,11,xi_1.00__kappa_25_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.0025,0.1,1.0,0.27,0.08,0.2,0.15,1.0
3,9,3.0,true,true,true,11,xi_1.00__kappa_30_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.003,0.1,1.0,0.27,0.08,0.2,0.15,1.0
4,13,3.0,true,true,true,11,xi_1.00__kappa_40_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.004,0.1,1.0,0.27,0.08,0.2,0.15,1.0
5,17,3.0,true,true,true,11,xi_1.00__kappa_50_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.005,0.1,1.0,0.27,0.08,0.2,0.15,1.0
6,21,3.0,true,true,true,11,xi_1.00__kappa_10_bp__gross_delta_200_bp__iota_0_bp__lambda_0.20__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.001,0.2,1.0,0.27,0.08,0.2,0.15,1.0
7,25,3.0,true,true,true,11,xi_1.00__kappa_25_bp__gross_delta_200_bp__iota_0_bp__lambda_0.20__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.0025,0.2,1.0,0.27,0.08,0.2,0.15,1.0
8,29,3.0,true,true,true,11,xi_1.00__kappa_30_bp__gross_delta_200_bp__iota_0_bp__lambda_0.20__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.003,0.2,1.0,0.27,0.08,0.2,0.15,1.0
9,33,3.0,true,true,true,11,xi_1.00__kappa_40_bp__gross_delta_200_bp__iota_0_bp__lambda_0.20__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.004,0.2,1.0,0.27,0.08,0.2,0.15,1.0


In [446]:
sum(diagdf[:count] .> 1)

555

In [420]:
bt = Batch.BatchObj()
bt = Batch.set_par_dict(bt, 8)

bt = Batch.set_comb_res_paths(bt)

# Create Directories
bt = Batch.mk_comb_res_dirs(bt)



# Check if results exist
# if Batch.check_batch_results(bt)
#     # Load SVM object
try
    svm = Batch.load_batch_results(bt)["svm"]

    # Check if Bond Pricing Surfaces exist
    surfs_exist = Batch.check_bpr_surfaces(svm)
catch
    # Create SVM object
    println("Unable to load Batch object file. Recomputing... ")
    svm = ModelObj.firm_constructor(bt.mi._svm_dict)
    surfs_exist = false
end 

Unable to load Batch object file. Recomputing... 
Setting initial vbl value to NaN
Setting initial vbh value to NaN
Setting initial c value to NaN
Setting initial p value to NaN


false

In [421]:
sum(diagdf[:count] .> 1)

260

In [344]:
conds = (abs.(diagdf[:, :m] .- 1) .< 1e-4) .* diagdf[:folder_created] .* (diagdf[:, :count] .< 11)

diagdf[conds, :]

,comb_num,m,folder_created,count,folder_name,V0,alpha,gross_delta,iota,kappa,lambda,mu_b,pi,r,sigmah,sigmal,xi
,Int64,Float64,Bool,Int64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,571,1.0,true,3,xi_1.00__kappa_50_bp__gross_delta_200_bp__iota_0_bp__lambda_0.30__sigmal_0.15__sigmah_0.30,100.0,0.6,0.02,0.0,0.005,0.3,1.0,0.27,0.08,0.3,0.15,1.0
2,576,1.0,true,3,xi_1.00__kappa_10_bp__gross_delta_200_bp__iota_0_bp__lambda_0.50__sigmal_0.15__sigmah_0.30,100.0,0.6,0.02,0.0,0.001,0.5,1.0,0.27,0.08,0.3,0.15,1.0
3,581,1.0,true,3,xi_1.00__kappa_25_bp__gross_delta_200_bp__iota_0_bp__lambda_0.50__sigmal_0.15__sigmah_0.30,100.0,0.6,0.02,0.0,0.0025,0.5,1.0,0.27,0.08,0.3,0.15,1.0
4,586,1.0,true,3,xi_1.00__kappa_30_bp__gross_delta_200_bp__iota_0_bp__lambda_0.50__sigmal_0.15__sigmah_0.30,100.0,0.6,0.02,0.0,0.003,0.5,1.0,0.27,0.08,0.3,0.15,1.0
5,591,1.0,true,3,xi_1.00__kappa_40_bp__gross_delta_200_bp__iota_0_bp__lambda_0.50__sigmal_0.15__sigmah_0.30,100.0,0.6,0.02,0.0,0.004,0.5,1.0,0.27,0.08,0.3,0.15,1.0
6,596,1.0,true,3,xi_1.00__kappa_50_bp__gross_delta_200_bp__iota_0_bp__lambda_0.50__sigmal_0.15__sigmah_0.30,100.0,0.6,0.02,0.0,0.005,0.5,1.0,0.27,0.08,0.3,0.15,1.0
7,601,1.0,true,3,xi_1.00__kappa_10_bp__gross_delta_200_bp__iota_0_bp__lambda_0.75__sigmal_0.15__sigmah_0.30,100.0,0.6,0.02,0.0,0.001,0.75,1.0,0.27,0.08,0.3,0.15,1.0
8,606,1.0,true,3,xi_1.00__kappa_25_bp__gross_delta_200_bp__iota_0_bp__lambda_0.75__sigmal_0.15__sigmah_0.30,100.0,0.6,0.02,0.0,0.0025,0.75,1.0,0.27,0.08,0.3,0.15,1.0
9,611,1.0,true,3,xi_1.00__kappa_30_bp__gross_delta_200_bp__iota_0_bp__lambda_0.75__sigmal_0.15__sigmah_0.30,100.0,0.6,0.02,0.0,0.003,0.75,1.0,0.27,0.08,0.3,0.15,1.0


In [399]:
sum(diagdf[:count])/11

350.09090909090907

In [345]:
diagdf[diagdf[:folder_created], :]

,comb_num,m,folder_created,count,folder_name,V0,alpha,gross_delta,iota,kappa,lambda,mu_b,pi,r,sigmah,sigmal,xi
,Int64,Float64,Bool,Int64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,1.0,true,11,xi_1.00__kappa_10_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.001,0.1,1.0,0.27,0.08,0.2,0.15,1.0
2,3,5.0,true,11,xi_1.00__kappa_10_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.001,0.1,1.0,0.27,0.08,0.2,0.15,1.0
3,4,7.0,true,11,xi_1.00__kappa_10_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.001,0.1,1.0,0.27,0.08,0.2,0.15,1.0
4,6,1.0,true,11,xi_1.00__kappa_25_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.0025,0.1,1.0,0.27,0.08,0.2,0.15,1.0
5,11,1.0,true,11,xi_1.00__kappa_30_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.003,0.1,1.0,0.27,0.08,0.2,0.15,1.0
6,16,1.0,true,11,xi_1.00__kappa_40_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.004,0.1,1.0,0.27,0.08,0.2,0.15,1.0
7,21,1.0,true,11,xi_1.00__kappa_50_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.005,0.1,1.0,0.27,0.08,0.2,0.15,1.0
8,26,1.0,true,11,xi_1.00__kappa_10_bp__gross_delta_200_bp__iota_0_bp__lambda_0.20__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.001,0.2,1.0,0.27,0.08,0.2,0.15,1.0
9,31,1.0,true,11,xi_1.00__kappa_25_bp__gross_delta_200_bp__iota_0_bp__lambda_0.20__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.0025,0.2,1.0,0.27,0.08,0.2,0.15,1.0


In [84]:
function get_results_path(bt, m)
    Batch.set_main_dir_path(bt)
    if bt.mi._svm_dict == nothing
        bt.mi.batch_res_dir = bt.dfn.res_dir    
    else
        Batch.set_batch_res_dir(bt)
    end

    return string(bt.mi.main_dir_path, "/", 
                  bt.mi.batch_res_dir, 
                  string(bt.dfn.mat_dir_prefix, @sprintf("%.1f", m)))
end

get_results_path (generic function with 1 method)

In [85]:
function load_eq_results(bt, path_file_name; 
                         col_types=vcat(fill(Float64, 16), [Bool], 
                                        fill(Float64, 14), [Bool]))
    dfs_list = [x for x in readdir(path_file_name) if 
                occursin(bt.dfn.eq_fd_cp_fn_prefix, x)]
    final = CSV.read(string(path_file_name, "/", dfs_list[1]), 
                     types=col_types)
    for df in dfs_list[2:end]
             final = vcat(final, 
                          CSV.read(string(path_file_name, "/", df), 
                                   types=col_types))
    end

    return sort!(final[final[:opt], :], :c) 
end

load_eq_results (generic function with 2 methods)

In [498]:
Batch.

Main.Batch.BatchStruct
  dfn: Main.Batch.DirFileNames
  dfc: Main.Batch.BatchDFColumns
  mi: Main.Batch.ModelInputs
  svm_param_values_dict: Dict{String,Array{T,1} where T}
  svm_coupon_grid: StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}
  pvb_tol_vec: Array{Float64}((4,)) [1.0e-6, 1.0e-5, 0.0001, 0.001]
  _common_params: Dict{String,Float64}
  _params_order: Array{String}((9,))
  _params_combs: Array{Array{Float64,1}}((1, 4, 1, 5, 1, 5, 1, 1, 6))
  coupon_grid: StepRange{Int64,Int64}


In [128]:
pfname = string(res_full_path, "/", resdirs[5])
pdf = load_eq_results(bt, pfname)

# optimal_k_struct(pdf)

,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,m,c,p,vb,debt_diff,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,abs_debt_diff,debt_per_diff,abs_debt_per_diff,debt,equity,firm_value,leverage,ROE,V0,r,alpha,pi,xi,opt
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool
1,0.02,0.02,0.0,0.001,0.1,0.3,0.15,1.0,1.0,1.0,12.3457,14.3574,-2.25814e-7,3.32203e-6,-0.000127942,0.000232537,false,4.4424e-7,2.25814e-7,-1.84425e-8,1.84425e-8,12.3457,84.7915,97.1372,12.7095,-3.26603,100.0,0.08,0.6,0.27,1.0,true
2,0.02,0.02,0.0,0.001,0.1,0.3,0.15,1.0,1.0,2.0,24.6914,28.7018,2.96501e-7,-6.1309e-6,-0.000184042,-8.40315e-5,false,-7.49976e-7,2.96501e-7,1.21022e-8,1.21022e-8,24.6914,74.0115,98.7028,25.0158,-1.72248,100.0,0.08,0.6,0.27,1.0,true
3,0.02,0.02,0.0,0.001,0.1,0.3,0.15,1.0,1.0,3.0,37.0372,43.0534,-4.05928e-8,-1.22753e-5,-0.00013055,0.00130605,false,4.21325e-7,4.05928e-8,-1.46181e-9,1.46181e-9,37.0372,62.2715,99.3087,37.295,-1.09797,100.0,0.08,0.6,0.27,1.0,true


In [115]:
res_full_path

"/mnt/ide0/home/artur/BondPricing/Julia/Results/m_1.0"

In [135]:
bt._params_order

9-element Array{String,1}:
 "mu_b"       
 "m"          
 "xi"         
 "kappa"      
 "gross_delta"
 "lambda"     
 "iota"       
 "sigmal"     
 "sigmah"     

In [134]:
bt._params_combs

1×1×1×5×1×5×1×1×6 Array{Array{Float64,1},9}:
[:, :, 1, 1, 1, 1, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.001, 0.02, 0.1, 0.0, 0.15, 0.2]

[:, :, 1, 2, 1, 1, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.0025, 0.02, 0.1, 0.0, 0.15, 0.2]

[:, :, 1, 3, 1, 1, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.003, 0.02, 0.1, 0.0, 0.15, 0.2]

[:, :, 1, 4, 1, 1, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.004, 0.02, 0.1, 0.0, 0.15, 0.2]

[:, :, 1, 5, 1, 1, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.005, 0.02, 0.1, 0.0, 0.15, 0.2]

[:, :, 1, 1, 1, 2, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.001, 0.02, 0.2, 0.0, 0.15, 0.2]

[:, :, 1, 2, 1, 2, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.0025, 0.02, 0.2, 0.0, 0.15, 0.2]

[:, :, 1, 3, 1, 2, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.003, 0.02, 0.2, 0.0, 0.15, 0.2]

[:, :, 1, 4, 1, 2, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.004, 0.02, 0.2, 0.0, 0.15, 0.2]

[:, :, 1, 5, 1, 2, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.005, 0.02, 0.2, 0.0, 0.15, 0.2]

[:, :, 1, 1, 1, 3, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.001, 0.02, 0.3, 0.0, 0.15, 0.2]

[:, :, 1, 2, 1, 3, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.00

In [129]:
resdirs[5]

"xi_1.00__kappa_10_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.30"

In [116]:
readdir(pfname)

9-element Array{String,1}:
 "batch_obj.jld"         
 "bond_surfaces.jld"     
 "debt_at_par_c_1.00.csv"
 "debt_at_par_c_2.00.csv"
 "debt_at_par_c_3.00.csv"
 "debt_at_par_c_4.00.csv"
 "eq_fd_c_1.00.csv"      
 "eq_fd_c_2.00.csv"      
 "eq_fd_c_3.00.csv"      

In [86]:
# Find Optimal Capital Structure
function optimal_k_struct(df; N=10^5)
    fvf = Dierckx.Spline1D(df[:c], df[:firm_value], k=3, bc="extrapolate")
    cgrid = range(minimum(df[:c]), stop=maximum(df[:c]), length=N)

    fdf = DataFrame()
    fdf[:c] = cgrid[argmax(fvf(cgrid))]
    fdf[:firm_value] = fvf(fdf[:c])
    
    vars1 = [:debt, :equity, :leverage, :ROE]
    vars2 = vcat(bt.dfc.debt_vars, bt.dfc.equity_vars)
    for var in vars1
        tmpf = Dierckx.Spline1D(df[:c], df[var], k=3, bc="extrapolate")
        fdf[var] = tmpf(fdf[:c])
    end
    
    # Get Parameter and Unique Values 
    unique_vars = [x for x in bt.dfc.dfcols if !(x in vcat(vars1, vars2))]
    for var in unique_vars
                    fdf[var] = unique(df[var])[1]
                end
    
    # Rearrange columns
    dfcols = [x for x in bt.dfc.dfcols if !(x in vcat(vars1, vars2))]
                            
    return fdf[dfcols]
end

optimal_k_struct (generic function with 1 method)

In [158]:
function process_results(res_full_path, resdir; 
                         col_types=vcat(fill(Float64, 16), [Bool], 
                                        fill(Float64, 14), [Bool]),  
                         N=10^5)
    pfname = string(res_full_path, "/", resdir)
    
    # Aggregate Results
    pdf = load_eq_results(bt, pfname, col_types=col_types)
    
    if size(pdf, 1) >= 8
        # Compute Optimal Capital Structure
        return optimal_k_struct(pdf; N=N)
    end
end

process_results (generic function with 1 method)

In [159]:
m = 1.
res_full_path = get_results_path(bt, m)
resdirs = [x for x in readdir(res_full_path)]


# resdir = resdirs[1]
#final = load_eq_results(bt, pfname)

LL = fetch(@spawn [process_results(res_full_path, dir) for dir in resdirs])

#optdf = sort(vcat(LL...), bt._params_order)

150-element Array{Union{Nothing, DataFrame},1}:
 1×18 DataFrame. Omitted printing of 12 columns
│ Row │ gross_delta │ delta   │ iota    │ kappa   │ lambda  │ sigmah  │
│     │ Float64     │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├─────┼─────────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 1   │ 0.02        │ 0.02    │ 0.0     │ 0.001   │ 0.1     │ 0.2     │
 1×18 DataFrame. Omitted printing of 12 columns
│ Row │ gross_delta │ delta   │ iota    │ kappa   │ lambda  │ sigmah  │
│     │ Float64     │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├─────┼─────────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 1   │ 0.02        │ 0.02    │ 0.0     │ 0.001   │ 0.1     │ 0.225   │
 1×18 DataFrame. Omitted printing of 12 columns
│ Row │ gross_delta │ delta   │ iota    │ kappa   │ lambda  │ sigmah  │
│     │ Float64     │ Float64 │ Float64 │ Float64 │ Float64 │ Float64 │
├─────┼─────────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 1   │ 0.02    

In [160]:
sum([x== nothing for x in LL])

66

In [225]:
    bt = Batch.set_par_dict(bt, comb_num)
    bt = Batch.set_comb_res_paths(bt)

Main.Batch.BatchStruct
  dfn: Main.Batch.DirFileNames
  dfc: Main.Batch.BatchDFColumns
  mi: Main.Batch.ModelInputs
  svm_param_values_dict: Dict{String,Array{T,1} where T}
  svm_coupon_grid: StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}
  pvb_tol_vec: Array{Float64}((4,)) [1.0e-6, 1.0e-5, 0.0001, 0.001]
  _common_params: Dict{String,Float64}
  _params_order: Array{String}((9,))
  _params_combs: Array{Array{Float64,1}}((1, 1, 1, 5, 1, 5, 1, 1, 6))
  coupon_grid: StepRange{Int64,Int64}


In [ ]:
Batch.mi

In [226]:
bt.mi.comb_res_path

"/mnt/ide0/home/artur/BondPricing/Julia/Results/m_1.0/xi_1.00__kappa_10_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20"

In [246]:
function diag_df(bt, comb_num)
    bt = Batch.set_par_dict(bt, comb_num)
    bt = Batch.set_comb_res_paths(bt)
    
    combdf = DataFrame(bt.mi._svm_dict)
    combdf[:comb_num] = comb_num
    combdf[:folder_name] = bt.mi.comb_res_dir

    # Count solutions:
    dfs_list = [x for x in readdir(bt.mi.comb_res_path) if 
                occursin(bt.dfn.eq_fd_cp_fn_prefix, x)]
    combdf[:count] = size(dfs_list, 1)
    
    return combdf
end

function comb_df(bt)
    combDFs = fetch(@spawn [diag_df(bt, comb) for comb in 
                            range(1, stop=size(bt._params_combs[:], 1))])
                
    col_order = vcat([:comb_num, :count, :folder_name], names(combDFs[1])[1:end-3])
    return vcat(combDFs...)[col_order]
end

comb_df (generic function with 2 methods)

In [247]:
tmp = comb_df(bt)

,comb_num,count,folder_name,V0,alpha,gross_delta,iota,kappa,lambda,m,mu_b,pi,r,sigmah,sigmal,xi
,Int64,Int64,String,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,1,11,xi_1.00__kappa_10_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.001,0.1,1.0,1.0,0.27,0.08,0.2,0.15,1.0
2,2,11,xi_1.00__kappa_25_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.0025,0.1,1.0,1.0,0.27,0.08,0.2,0.15,1.0
3,3,11,xi_1.00__kappa_30_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.003,0.1,1.0,1.0,0.27,0.08,0.2,0.15,1.0
4,4,11,xi_1.00__kappa_40_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.004,0.1,1.0,1.0,0.27,0.08,0.2,0.15,1.0
5,5,11,xi_1.00__kappa_50_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.005,0.1,1.0,1.0,0.27,0.08,0.2,0.15,1.0
6,6,11,xi_1.00__kappa_10_bp__gross_delta_200_bp__iota_0_bp__lambda_0.20__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.001,0.2,1.0,1.0,0.27,0.08,0.2,0.15,1.0
7,7,11,xi_1.00__kappa_25_bp__gross_delta_200_bp__iota_0_bp__lambda_0.20__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.0025,0.2,1.0,1.0,0.27,0.08,0.2,0.15,1.0
8,8,11,xi_1.00__kappa_30_bp__gross_delta_200_bp__iota_0_bp__lambda_0.20__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.003,0.2,1.0,1.0,0.27,0.08,0.2,0.15,1.0
9,9,11,xi_1.00__kappa_40_bp__gross_delta_200_bp__iota_0_bp__lambda_0.20__sigmal_0.15__sigmah_0.20,100.0,0.6,0.02,0.0,0.004,0.2,1.0,1.0,0.27,0.08,0.2,0.15,1.0


15-element Array{Symbol,1}:
 :combination
 :folder_name
 :V0         
 :alpha      
 :gross_delta
 :iota       
 :kappa      
 :lambda     
 :m          
 :mu_b       
 :pi         
 :r          
 :sigmah     
 :sigmal     
 :xi         

In [166]:
comb_num = 1
svm_dict = Dict()
for par in bt._params_order
    svm_dict[par] = bt._params_combs[comb_num][findall(x-> x==par, 
                                                       bt._params_order)[1]]
end


#bt.mi._svm_dict[par] = bt._params_combs[comb_num][findall(x-> x==par, bt._params_order)[1]]
svm_dict

Dict{Any,Any} with 9 entries:
  "sigmal"      => 0.15
  "m"           => 1.0
  "kappa"       => 0.001
  "iota"        => 0.0
  "lambda"      => 0.1
  "xi"          => 1.0
  "sigmah"      => 0.2
  "gross_delta" => 0.02
  "mu_b"        => 1.0

In [ ]:
function diagnose(path_dir)
    return 

In [ ]:
[ for dir in resdirs

In [164]:
bt._params_combs

1×1×1×5×1×5×1×1×6 Array{Array{Float64,1},9}:
[:, :, 1, 1, 1, 1, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.001, 0.02, 0.1, 0.0, 0.15, 0.2]

[:, :, 1, 2, 1, 1, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.0025, 0.02, 0.1, 0.0, 0.15, 0.2]

[:, :, 1, 3, 1, 1, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.003, 0.02, 0.1, 0.0, 0.15, 0.2]

[:, :, 1, 4, 1, 1, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.004, 0.02, 0.1, 0.0, 0.15, 0.2]

[:, :, 1, 5, 1, 1, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.005, 0.02, 0.1, 0.0, 0.15, 0.2]

[:, :, 1, 1, 1, 2, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.001, 0.02, 0.2, 0.0, 0.15, 0.2]

[:, :, 1, 2, 1, 2, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.0025, 0.02, 0.2, 0.0, 0.15, 0.2]

[:, :, 1, 3, 1, 2, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.003, 0.02, 0.2, 0.0, 0.15, 0.2]

[:, :, 1, 4, 1, 2, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.004, 0.02, 0.2, 0.0, 0.15, 0.2]

[:, :, 1, 5, 1, 2, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.005, 0.02, 0.2, 0.0, 0.15, 0.2]

[:, :, 1, 1, 1, 3, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.001, 0.02, 0.3, 0.0, 0.15, 0.2]

[:, :, 1, 2, 1, 3, 1, 1, 1] =
 [1.0, 1.0, 1.0, 0.00

In [ ]:
"

In [95]:
pfname = string(res_full_path, "/", resdirs[2])

"/mnt/ide0/home/artur/BondPricing/Julia/Results/m_1.0/xi_1.00__kappa_10_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.23"

In [101]:
resdirs[i]

"xi_1.00__kappa_10_bp__gross_delta_200_bp__iota_0_bp__lambda_0.10__sigmal_0.15__sigmah_0.35"

In [140]:
process_results(res_full_path, resdirs[i])

UndefVarError: UndefVarError: len not defined

In [106]:
pfname = string(res_full_path, "/", resdirs[i])

# Aggregate Results
pdf = load_eq_results(bt, pfname)



,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,m,c,p,vb,debt_diff,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,abs_debt_diff,debt_per_diff,abs_debt_per_diff,debt,equity,firm_value,leverage,ROE,V0,r,alpha,pi,xi,opt
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool
1,0.02,0.02,0.0,0.001,0.1,0.35,0.15,1.0,1.0,1.0,12.3457,14.861,-2.25814e-7,-1.60427e-5,-0.000237637,0.000304363,false,-4.68425e-7,2.25814e-7,-1.84425e-8,1.84425e-8,12.3457,80.936,93.2817,13.2348,-7.66456,100.0,0.08,0.6,0.27,1.0,true
2,0.02,0.02,0.0,0.001,0.1,0.35,0.15,1.0,1.0,2.0,24.6914,29.7176,2.96501e-7,-1.57156e-6,-0.000274486,0.000458759,false,1.11246e-7,2.96501e-7,1.21022e-8,1.21022e-8,24.6914,68.8365,93.5279,26.4,-8.5941,100.0,0.08,0.6,0.27,1.0,true
3,0.02,0.02,0.0,0.001,0.1,0.35,0.15,1.0,1.0,3.0,37.0323,44.5717,2.66547e-7,1.41417e-5,-0.000133957,0.000861129,false,-7.88881e-7,2.66547e-7,2.40296e-8,2.40296e-8,37.0323,56.5976,93.6299,39.5518,-10.1165,100.0,0.08,0.6,0.27,1.0,true


In [100]:
i = 6
process_results(res_full_path, resdirs[i])

ErrorException: k must be less than length(x)

In [96]:
resdir[2]

'i': ASCII/Unicode U+0069 (category Ll: Letter, lowercase)

In [54]:
vars1 = [:debt, :equity, :leverage, :ROE]
vars2 = [:eq_deriv, :eq_deriv_min_val, :eq_min_val, 
          :eq_negative, :eq_vb, :abs_debt_diff, 
          :debt_per_diff, :abs_debt_per_diff]

[x for x in bt.dfc.dfcols if !(x in vcat(vars1, vars2))]

19-element Array{Symbol,1}:
 :gross_delta
 :delta      
 :iota       
 :kappa      
 :lambda     
 :sigmah     
 :sigmal     
 :mu_b       
 :m          
 :c          
 :p          
 :vb         
 :debt_diff  
 :firm_value 
 :V0         
 :r          
 :alpha      
 :pi         
 :xi         

In [53]:
[]

```
occursin(needle::Union{AbstractString,Regex,AbstractChar}, haystack::AbstractString)
```

Determine whether the first argument is a substring of the second. If `needle` is a regular expression, checks whether `haystack` contains a match.

# Examples

```jldoctest
julia> occursin("Julia", "JuliaLang is pretty cool!")
true

julia> occursin('a', "JuliaLang is pretty cool!")
true

julia> occursin(r"a.a", "aba")
true

julia> occursin(r"a.a", "abba")
false
```


31-element Array{Symbol,1}:
 :gross_delta      
 :delta            
 :iota             
 :kappa            
 :lambda           
 :sigmah           
 :sigmal           
 :mu_b             
 :m                
 :c                
 :p                
 :vb               
 :debt_diff        
 :eq_deriv         
 :eq_deriv_min_val 
 :eq_min_val       
 :eq_negative      
 :eq_vb            
 :abs_debt_diff    
 :debt_per_diff    
 :abs_debt_per_diff
 :debt             
 :equity           
 :firm_value       
 :leverage         
 :ROE              
 :V0               
 :r                
 :alpha            
 :pi               
 :xi               

In [33]:
resdir = resdirs[1]

pfname = string(res_full_path, "/", resdir)

final = load_eq_results(bt, pfname)

,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,m,c,p,vb,debt_diff,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,abs_debt_diff,debt_per_diff,abs_debt_per_diff,debt,equity,firm_value,leverage,ROE,V0,r,alpha,pi,xi,opt
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool
1,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,1.0,12.3457,13.3148,-2.25814e-7,-6.62849e-6,0.000925969,0.00224349,false,1.89095e-8,2.25814e-7,-1.84425e-8,1.84425e-8,12.3457,90.5439,102.89,11.999,3.29656,100.0,0.08,0.6,0.27,1.0,true
2,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,2.0,24.6914,26.683,2.96501e-7,-2.82451e-6,0.00111681,0.00391515,false,2.03016e-8,2.96501e-7,1.21022e-8,1.21022e-8,24.6914,81.2472,105.939,23.3072,7.88568,100.0,0.08,0.6,0.27,1.0,true
3,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,3.0,37.037,40.0458,-2.96399e-7,3.66267e-6,0.00040668,0.00543088,false,4.62569e-8,2.96399e-7,-9.60848e-9,9.60848e-9,37.037,71.542,108.579,34.1107,13.6255,100.0,0.08,0.6,0.27,1.0,true
4,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,4.0,49.378,53.388,-1.31148e-8,-2.07513e-5,0.00101936,0.00713596,false,-1.10808e-6,1.31148e-8,-5.28369e-10,5.28369e-10,49.378,60.6433,110.021,44.8804,19.7965,100.0,0.08,0.6,0.27,1.0,true
5,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,5.0,61.4221,66.3909,1.48016e-6,2.22711e-6,1.92259e-6,0.000709966,false,-5.82058e-13,1.48016e-6,2.52122e-8,2.52122e-8,61.4221,47.5058,108.928,56.3878,23.1426,100.0,0.08,0.6,0.27,1.0,true
6,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,6.0,69.4148,74.7015,-6.79703e-6,1.29078e-5,1.2877e-5,0.00081629,false,0.0,6.79703e-6,-9.07232e-8,9.07232e-8,69.4148,37.0341,106.449,65.2095,21.085,100.0,0.08,0.6,0.27,1.0,true
7,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,7.0,73.3653,78.3378,1.59401e-5,6.7388e-6,6.83365e-6,0.001441,false,0.0,1.59401e-5,2.1689e-7,2.1689e-7,73.3653,32.1875,105.553,69.5058,20.8482,100.0,0.08,0.6,0.27,1.0,true
8,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,8.0,75.9778,80.4168,2.92583e-5,5.67454e-6,5.65242e-6,0.0020283,false,0.0,2.92583e-5,3.86176e-7,3.86176e-7,75.9778,29.5037,105.482,72.0295,22.8186,100.0,0.08,0.6,0.27,1.0,true
9,0.02,0.02,0.0,0.001,0.1,0.2,0.15,1.0,1.0,9.0,78.0228,81.8296,2.80976e-5,-8.08075e-6,-8.08149e-6,0.00186128,false,0.0,2.80976e-5,4.61623e-7,4.61623e-7,78.0228,27.7844,105.807,73.7405,26.4236,100.0,0.08,0.6,0.27,1.0,true


In [31]:
[x for x in readdir(pfname) if 
                occursin(bt.dfn.eq_fd_cp_fn_prefix, x)]

11-element Array{String,1}:
 "eq_fd_c_1.00.csv" 
 "eq_fd_c_10.00.csv"
 "eq_fd_c_11.00.csv"
 "eq_fd_c_2.00.csv" 
 "eq_fd_c_3.00.csv" 
 "eq_fd_c_4.00.csv" 
 "eq_fd_c_5.00.csv" 
 "eq_fd_c_6.00.csv" 
 "eq_fd_c_7.00.csv" 
 "eq_fd_c_8.00.csv" 
 "eq_fd_c_9.00.csv" 

In [13]:
bt.mi._svm_dict == nothing

true

In [68]:
# Import and Filter Finite Differences Results
col_types = vcat(fill(Float64, 16), [Bool], fill(Float64, 14), [Bool])
dfs_list = [x for x in readdir(bt.mi.comb_res_path) if occursin(bt.dfn.eq_fd_cp_fn_prefix, x)]

final = CSV.read(string(bt.mi.comb_res_path, "/", dfs_list[1]), types=col_types)
for df in dfs_list[2:end]
         final = vcat(final, CSV.read(string(bt.mi.comb_res_path, "/", df), types=col_types))
end
            
fdf = sort!(final[final[:opt], :], :c)

,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,m,c,p,vb,debt_diff,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,abs_debt_diff,debt_per_diff,abs_debt_per_diff,debt,equity,firm_value,leverage,ROE,V0,r,alpha,pi,xi,opt
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool
1,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,1.0,10.5263,11.7309,-1.66729e-7,-2.42744e-6,7.12015e-6,0.000637466,false,6.46128e-7,1.66729e-7,-1.60162e-8,1.60162e-8,10.5263,91.4235,101.95,10.325,2.17921,100.0,0.08,0.6,0.27,1.0,true
2,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,2.0,21.0526,23.516,-2.74032e-7,-3.81735e-6,-2.40597e-6,0.00144636,false,3.03687e-7,2.74032e-7,-1.29069e-8,1.29069e-8,21.0526,82.0078,103.06,20.4275,3.87655,100.0,0.08,0.6,0.27,1.0,true
3,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,3.0,31.5789,35.3262,-1.67226e-7,3.53202e-6,3.1164e-6,0.00240565,false,6.22058e-8,1.67226e-7,-6.9371e-9,6.9371e-9,31.5789,72.2474,103.826,30.4152,5.59235,100.0,0.08,0.6,0.27,1.0,true
4,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,4.0,42.088,47.0799,-2.17836e-7,-1.33136e-6,-2.16175e-6,0.0032007,false,3.16342e-7,2.17836e-7,3.72757e-9,3.72757e-9,42.088,61.6904,103.778,40.5556,6.5245,100.0,0.08,0.6,0.27,1.0,true
5,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,5.0,52.3457,58.5445,-9.12043e-7,1.05496e-5,1.13671e-5,0.00145626,false,-2.37215e-6,9.12043e-7,2.20593e-10,2.20593e-10,52.3457,49.6294,101.975,51.3319,4.14449,100.0,0.08,0.6,0.27,1.0,true
6,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,6.0,60.0294,66.9727,-2.62359e-6,-5.01528e-6,-5.00939e-6,-7.9414e-5,false,9.14345e-12,2.62359e-6,-1.55961e-8,1.55961e-8,60.0294,39.468,99.4974,60.3326,-1.25742,100.0,0.08,0.6,0.27,1.0,true
7,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,7.0,64.4443,71.552,8.73021e-6,8.38613e-6,8.38613e-6,0.000166169,false,0.0,8.73021e-6,3.01143e-8,3.01143e-8,64.4444,33.7399,98.1842,65.6362,-5.1069,100.0,0.08,0.6,0.27,1.0,true
8,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,8.0,67.3156,74.3132,8.79634e-6,4.37998e-6,4.37998e-6,0.00179035,false,0.0,8.79634e-6,-7.40658e-8,7.40658e-8,67.3156,30.3741,97.6897,68.9076,-7.0685,100.0,0.08,0.6,0.27,1.0,true
9,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,9.0,69.4889,76.2619,1.78414e-5,4.63238e-6,4.63238e-6,0.000816766,false,0.0,1.78414e-5,9.16431e-8,9.16431e-8,69.4889,28.0814,97.5703,71.2193,-7.96327,100.0,0.08,0.6,0.27,1.0,true


In [73]:
# Find Optimal Capital Structure
N=10^5
fvf = Dierckx.Spline1D(fdf[:c], fdf[:firm_value], k=3, bc="extrapolate")
cgrid = range(minimum(fdf[:c]), stop=maximum(fdf[:c]), length=N)

optc = cgrid[argmax(fvf(cgrid))]
optFV = fvf(optc)

103.97981964146865

In [44]:
tmp = CSV.read(string(bt.mi.comb_res_path, "/", dfs_list[1])) #, types=col_types) 
tmp2 = CSV.read(string(bt.mi.comb_res_path, "/", dfs_list[2]))

,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,m,c,p,vb,debt_diff,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,abs_debt_diff,debt_per_diff,abs_debt_per_diff,debt,equity,firm_value,leverage,ROE,V0,r,alpha,pi,xi,opt
,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Bool⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Float64⍰,Bool⍰
1,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,10.0,60.4661,65.0009,1.96254,-7.92992e-6,-7.88288e-6,0.000809549,false,0.0,1.96254,0.0324569,0.0324569,62.4287,47.9599,110.389,56.5536,27.6502,100.0,0.08,0.6,0.27,1.0,false
2,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,10.0,61.5893,66.3364,1.87049,-8.89988e-6,-8.89964e-6,3.58055e-5,false,0.0,1.87049,0.0303704,0.0303704,63.4598,45.8152,109.275,58.0735,25.383,100.0,0.08,0.6,0.27,1.0,false
3,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,10.0,62.7125,67.6666,1.76533,-1.68586e-6,-1.68586e-6,0.00115234,false,0.0,1.76533,0.0281496,0.0281496,64.4779,43.6454,108.123,59.6337,22.8682,100.0,0.08,0.6,0.27,1.0,false
4,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,10.0,63.8358,68.9737,1.64237,-1.03365e-5,-1.03365e-5,0.00225679,false,0.0,1.64237,0.025728,0.025728,65.4781,41.4863,106.964,61.2149,20.1739,100.0,0.08,0.6,0.27,1.0,false
5,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,10.0,64.959,70.3011,1.49828,4.43594e-6,4.43594e-6,0.000988805,false,0.0,1.49828,0.023065,0.023065,66.4572,39.258,105.715,62.8644,17.0386,100.0,0.08,0.6,0.27,1.0,false
6,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,10.0,66.0822,71.6338,1.32835,1.35781e-7,1.35781e-7,0.000393617,false,0.0,1.32835,0.0201015,0.0201015,67.4105,36.9934,104.404,64.5671,13.5132,100.0,0.08,0.6,0.27,1.0,false
7,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,10.0,67.2054,72.9448,1.12218,5.58395e-6,5.58395e-6,0.00199436,false,0.0,1.12218,0.0166977,0.0166977,68.3276,34.746,103.074,66.2901,9.70417,100.0,0.08,0.6,0.27,1.0,false
8,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,10.0,68.3286,74.2695,0.879715,-7.43459e-6,-7.43459e-6,0.00147966,false,0.0,0.879715,0.0128748,0.0128748,69.2083,32.4548,101.663,68.0761,5.40117,100.0,0.08,0.6,0.27,1.0,false
9,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,10.0,69.4518,75.6007,0.589344,1.22864e-6,1.22864e-6,0.000519649,false,0.0,0.589344,0.00848566,0.00848566,70.0411,30.1408,100.182,69.9139,0.607378,100.0,0.08,0.6,0.27,1.0,false


In [54]:
DataFrame(Array(col_types))

UndefRefError: UndefRefError: access to undefined reference

In [47]:
DataFrame()

""
""


In [48]:
vcat(tmp, DataFrame())

ArgumentError: ArgumentError: column(s) gross_delta, delta, iota, kappa, lambda, sigmah, sigmal, mu_b, m, c, p, vb, debt_diff, eq_deriv, eq_deriv_min_val, eq_min_val, eq_negative, eq_vb, abs_debt_diff, debt_per_diff, abs_debt_per_diff, debt, equity, firm_value, leverage, ROE, V0, r, alpha, pi, xi and opt are missing from argument(s) 2

In [443]:
string(bt.mi.comb_res_path, "/", dfs_list[1])

"/mnt/ide0/home/artur/BondPricing/Julia/Results/Tests/m_1.0/xi_1.00__kappa_150_bp__gross_delta_200_bp__iota_0_bp__lambda_0.30__sigmal_0.20__sigmah_0.20/eq_fd_c_1.00.jld"

In [442]:
load(string(bt.mi.comb_res_path, "/", dfs_list[1]))

ErrorException: unrecognized HDF5 datatype class 4

In [436]:
filter(x->contains(x,bt.dfn.eq_fd_cp_fn_prefix), readdir(bt.mi.comb_res_path))

UndefVarError: UndefVarError: contains not defined

In [84]:
eqfinal_sol = sort!(vcat(eqdf_final[bt.dfc.dfcols], res[bt.dfc.dfcols]), [:p])

eqfinal_sol[:opt] = (abs.(eqfinal_sol[:abs_debt_diff] .- 
                     minimum(eqfinal_sol[:abs_debt_diff])) .< 1e-4) .* 
                    (abs.(eqfinal_sol[:eq_deriv]) .< 1e-4)

eqfinal_sol

,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,m,c,p,vb,debt_diff,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,abs_debt_diff,debt_per_diff,abs_debt_per_diff,debt,equity,firm_value,leverage,ROE,V0,r,alpha,pi,xi,opt
,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float32,Float32,Float32,Float32,Float32,Bool
1,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,2.0,20.2187,22.5321,0.0383904,5.0437e-6,2.73432e-6,0.00164775,false,6.09102e-7,0.0383904,0.00189876,0.00189876,20.2571,82.9778,103.235,19.6223,4.05661,100.0,0.08,0.6,0.27,1.0,false
2,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,2.0,20.5511,22.9242,0.02309,-7.07755e-6,4.66086e-6,0.00169912,false,-2.23847e-6,0.02309,0.00112354,0.00112354,20.5742,82.5915,103.166,19.9428,3.98568,100.0,0.08,0.6,0.27,1.0,false
3,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,2.0,20.8834,23.3163,0.00778956,-1.68925e-5,-1.8649e-5,0.00175788,false,6.01445e-7,0.00778956,0.000373002,0.000373002,20.8912,82.2049,103.096,20.2638,3.91379,100.0,0.08,0.6,0.27,1.0,false
4,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,2.0,21.0526,23.5161,-2.74032e-7,2.66739e-6,-1.71073e-6,0.00179129,false,7.80069e-7,2.74032e-7,-1.29069e-8,1.29069e-8,21.0526,82.0076,103.06,20.4275,3.87629,100.0,0.08,0.6,0.27,1.0,true
5,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,2.0,21.2158,23.7087,-0.00751087,1.66272e-5,1.50645e-5,0.00182392,false,6.01011e-7,0.00751087,-0.000354023,0.000354023,21.2083,81.8174,103.026,20.5854,3.84005,100.0,0.08,0.6,0.27,1.0,false
6,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,2.0,21.5481,24.1007,-0.0228113,-1.76675e-5,-4.03347e-6,0.00188559,false,1.08495e-6,0.0228113,-0.00105862,0.00105862,21.5253,81.4308,102.956,20.9073,3.76703,100.0,0.08,0.6,0.27,1.0,false
7,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,2.0,21.8805,24.4929,-0.0381117,-8.72211e-6,2.08788e-6,0.00193996,false,-3.62949e-6,0.0381117,-0.00174181,0.00174181,21.8424,81.0433,102.886,21.2298,3.69211,100.0,0.08,0.6,0.27,1.0,false


In [ ]:
function save_eq_results(res, df, eq_fd_cp_path_fname)
    sol = sort!(vcat(df[bt.dfc.dfcols], res[bt.dfc.dfcols]), [:p])

    # Identify solution
    sol[:opt] = (abs.(sol[:abs_debt_diff] .- 
                         minimum(sol[:abs_debt_diff])) .< 1e-4) .* 
                        (abs.(sol[:eq_deriv]) .< 1e-4)
    
    save(eq_fd_cp_path_fname, sol, "df")
end

In [85]:
string(bt.mi.comb_res_path, "/", 
       bt.dfn.eq_fd_cp_fn_prefix, 

"/mnt/ide0/home/artur/BondPricing/Julia/Results/Tests/m_1.0/xi_1.00__kappa_150_bp__gross_delta_200_bp__iota_0_bp__lambda_0.30__sigmal_0.20__sigmah_0.20"

In [86]:
bt.dfn.eq_fd_cp_fn_prefix

"eq_fd_c_"

In [161]:
vbOpt = vbf(pOpt)

23.516068958228008

In [134]:
count["cvml"]

Dict{String,Any} with 5 entries:
  "cvm_vb_fun" => Spline1D(knots=[5.0,13.9474 … 81.0526,90.0] (18 elements), k=…
  "cvm_dff"    => Spline1D(knots=[5.0,13.9474 … 81.0526,90.0] (18 elements), k=…
  "cvm_vbOpt"  => 23.636
  "cvm_debt"   => [5.73899, 10.0067, 14.2745, 18.5422, 22.8099, 27.0777, 31.345…
  "cvm_pOpt"   => 21.0496

In [221]:
# function compute_debt_diffs(df)
#     aggP = [AnalyticFunctions.get_agg_p(svm, p=p) for p in df[:p]]
#     df[:abs_debt_diff] = abs.(df[:debt] .- aggP)
#     df[:debt_per_diff] = (df[:debt] .- aggP) ./ aggP
#     df[:abs_debt_per_diff] = abs.(df[:debt_per_diff])
    
#     return df
# end

# function batch_II_rearrange_cols(df; cols=[])
#     if isempty(cols)
#         main_params = [:delta, :kappa, :lambda, :sigmah, :sigmal]
#         k_struct_params = [:c, :p, :vb]
#         fixed_params = [:mu_b, :m, :V0, :r, :alpha , :pi, :xi]
#         result_vars = [:abs_debt_diff, 
#                    :eq_deriv,       
#                    :eq_deriv_min_val, 
#                    :eq_min_val,    
#                    :eq_negative,     
#                    :eq_vb, 
#                    :debt_per_diff,
#                    :abs_debt_per_diff]
#         share_values = [:debt, :equity, :firm_value, :leverage]

#         cols = vcat(main_params, k_struct_params, result_vars, share_values, fixed_params)
#     end
    
#     return df[cols]
# end

batch_II_rearrange_cols (generic function with 1 method)

In [348]:
using Parameters

In [223]:
eqdf_all = compute_debt_diffs(results)
eqdf_all = batch_II_rearrange_cols(eqdf)

,delta,kappa,lambda,sigmah,sigmal,c,p,vb,abs_debt_diff,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,debt_per_diff,abs_debt_per_diff,debt,equity,firm_value,leverage,mu_b,m,V0,r,alpha,pi,xi
,Float32,Float32,Float32,Float32,Float32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float32,Float32,Float32,Float32,Float32
1,0.02,0.015,0.3,0.2,0.2,2.0,20.2187,16.2011,0.0383904,-1.12448,-1.12448,-0.298282,true,0.0,0.00189876,0.00189876,20.2571,110.752,131.009,15.4624,1.0,1.0,100.0,0.08,0.6,0.27,1.0
2,0.02,0.015,0.3,0.2,0.2,2.0,20.2187,17.7066,0.0383904,-0.818833,-0.818833,-0.204069,true,0.0,0.00189876,0.00189876,20.2571,102.355,122.612,16.5213,1.0,1.0,100.0,0.08,0.6,0.27,1.0
3,0.02,0.015,0.3,0.2,0.2,2.0,20.2187,19.2121,0.0383904,-0.552998,-0.552998,-0.119109,true,0.0,0.00189876,0.00189876,20.2571,95.27,115.527,17.5345,1.0,1.0,100.0,0.08,0.6,0.27,1.0
4,0.02,0.015,0.3,0.2,0.2,2.0,20.2187,20.7177,0.0383904,-0.305808,-0.305808,-0.0462776,false,0.0,0.00189876,0.00189876,20.2571,89.2107,109.468,18.5051,1.0,1.0,100.0,0.08,0.6,0.27,1.0
5,0.02,0.015,0.3,0.2,0.2,2.0,20.2187,22.2232,0.0383904,-0.0520248,-0.0520248,-0.00170091,false,0.0,0.00189876,0.00189876,20.2571,83.9677,104.225,19.436,1.0,1.0,100.0,0.08,0.6,0.27,1.0
6,0.02,0.015,0.3,0.2,0.2,2.0,20.2187,23.7287,0.0383904,0.193329,0.193329,0.00643404,false,0.0,0.00189876,0.00189876,20.2571,79.3849,99.642,20.3299,1.0,1.0,100.0,0.08,0.6,0.27,1.0
7,0.02,0.015,0.3,0.2,0.2,2.0,20.2187,25.2342,0.0383904,0.401146,0.401146,0.0133533,false,0.0,0.00189876,0.00189876,20.2571,75.3434,95.6005,21.1893,1.0,1.0,100.0,0.08,0.6,0.27,1.0
8,0.02,0.015,0.3,0.2,0.2,2.0,20.2187,26.7397,0.0383904,0.570381,0.570381,0.0193524,false,0.0,0.00189876,0.00189876,20.2571,71.7513,92.0084,22.0166,1.0,1.0,100.0,0.08,0.6,0.27,1.0
9,0.02,0.015,0.3,0.2,0.2,2.0,20.2187,28.2452,0.0383904,0.717031,0.717031,0.0248473,false,0.0,0.00189876,0.00189876,20.2571,68.5359,88.793,22.8138,1.0,1.0,100.0,0.08,0.6,0.27,1.0


In [61]:
tmp = Dict()
for par in vcat(bt.dfc.main_params, [:c], bt.dfc.fixed_params)
    tmp[par] = unique(eqdf_all[par])[1]
end

tmp

Dict{Any,Any} with 13 entries:
  :alpha       => 0.6
  :c           => 2.0
  :delta       => 0.02
  :pi          => 0.27
  :lambda      => 0.3
  :gross_delta => 0.02
  :V0          => 100.0
  :kappa       => 0.015
  :sigmal      => 0.2
  :iota        => 0.0
  :xi          => 1.0
  :r           => 0.08
  :sigmah      => 0.2

In [69]:
DataFrame(tmp)

,V0,alpha,c,delta,gross_delta,iota,kappa,lambda,pi,r,sigmah,sigmal,xi
,Float32,Float32,Float64,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float32
1,100.0,0.6,2.0,0.02,0.02,0.0,0.015,0.3,0.27,0.08,0.2,0.2,1.0


In [70]:
function search_eq_deriv(svm, pOpt, vbOpt; params_dict=Dict(),
                    lower_adj=.85, upper_adj=1.25, grid_size=3)
    if df==nothing
        df = DataFrame()

        # Create VB grid
        vbgrid = range(lower_adj, stop=upper_adj, length=grid_size) .* vbOpt
        vbgrid = unique(vcat(vbgrid, [vbOpt]))

        vbb = vbgrid
    else
        vbgrid = unique(vcat(df[:vb], [vbOpt]))
        vbb = [vb for vb in vbgrid if !(vb in df[:vb])]
    end
                
    LL = fetch(@spawn [EqFinDiff.eq_fd(svm, vb; mu_b=svm.mu_b, c=svm.c, p=pOpt) 
                      for vb in vbb])
    df = vcat(LL..., DataFrame(params_dict))

    return sort!(df, :vb)
end

search_eq_deriv (generic function with 1 method)

In [111]:
using Dierckx

function eq_deriv_root_search(svm, df, p; mu_b=NaN, c=NaN, N=10^5, params_dict=Dict())
    if isnan(mu_b)
        mu_b=svm.mu_b 
    end
    
    if isnan(c)
        c=svm.c
    end
    
    # Create DataFrame
    res = DataFrame()
#     for par in vcat(bt.dfc.main_params, [:c], bt.dfc.fixed_params)
#         res[par] = unique(eqdf_all[par])[1]
#     end
    
#     params_dict = Dict()
#     for par in vcat(main_params, [:c], fixed_params)
#         params_dict[par] = unique(eqdf_all[par])[1]
#     end
    
    # Filter DataFrame
    fdf = df[abs.(df[:p] .- p) .< 1e-4, :]
    
    # Interpolate Values
    eq_deriv_fun = Dierckx.Spline1D(fdf[:vb], fdf[:eq_deriv], k=3, bc="extrapolate")
    debt_fun = Dierckx.Spline1D(fdf[:vb], fdf[:debt], k=3, bc="extrapolate")
    
    # Refine Grid of VB values
    ref_vbgrid = range(minimum(fdf[:vb]), stop=maximum(fdf[:vb]), length=N)
    
    # Compute optimal VB:
    vbOpt = ref_vbgrid[argmin(abs.(eq_deriv_fun(ref_vbgrid)))]
    res[:vb] = vbOpt
    # Share Values
    
    # Debt Values
    res[:p] = p
    res[:debt] = debt_fun(vbOpt) 
    
    res = Batch.debt_at_par_diffs(svm, res, :p)  
#     aggP = AnalyticFunctions.get_agg_p(svm, p=p)
#     debt_diff = debtOpt - aggP
#     debt_per_diff = debt_diff/aggP
#     abs_debt_diff = abs(debt_diff)
#     abs_debt_per_diff = abs(debt_per_diff)
    
    # Equity Values
    res[:eq_deriv] = eq_deriv_fun(vbOpt)
    
    for var in [:equity, :eq_deriv_min_val, :eq_min_val, :eq_vb ]
        tmpf = Dierckx.Spline1D(fdf[:vb], fdf[var], k=3, bc="extrapolate")
        res[var] = tmpf(vbOpt)
    end
    
    res[:eq_negative] = (res[:eq_min_val] .< .05)
    
    res[:firm_value] = res[:debt] .+ res[:equity]
    
    res[:leverage] = res[:debt]./res[:firm_value]
    res[:ROE] = res[:equity]./(res[:firm_value] - res[:debt])
    
#     if !isempty(params_dict)
#         for key in keys(params_dict)
#             res[key] = params_dict[key]
#         end
#     end
    
    return res #vcat(res, DataFrame(params_dict))
end

eq_deriv_root_search (generic function with 1 method)

In [115]:
params_dict

Dict{Any,Any} with 15 entries:
  :alpha       => 0.6
  :c           => 2.0
  :delta       => 0.02
  :pi          => 0.27
  :lambda      => 0.3
  :gross_delta => 0.02
  :V0          => 100.0
  :kappa       => 0.015
  :sigmal      => 0.2
  :iota        => 0.0
  :mu_b        => 1.0
  :xi          => 1.0
  :r           => 0.08
  :m           => 1.0
  :sigmah      => 0.2

In [116]:
bt.dfc.main_params

7-element Array{Symbol,1}:
 :gross_delta
 :delta      
 :iota       
 :kappa      
 :lambda     
 :sigmah     
 :sigmal     

In [124]:
params_dict = Dict()
for par in vcat(bt.dfc.main_params, [:mu_b, :m, :c], bt.dfc.fixed_params)
    params_dict[par] = unique(eqdf_all[par])[1]
end

# eq_deriv_root_search(svm, eqdf_all, p; params_dict=params_dict)

tmp1 = fetch(@spawn [eq_deriv_root_search(svm, eqdf_all, p; params_dict=params_dict) 
        for p in unique(eqdf_all[:p])])

tmp2 = hcat(vcat(tmp1...), repeat(DataFrame(params_dict), outer=size(tmp1, 1)))[bt.dfc.dfcols]
# batch_II_rearrange_cols(vcat(res...))

,gross_delta,delta,iota,kappa,lambda,sigmah,sigmal,mu_b,m,c,p,vb,abs_debt_diff,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,debt_per_diff,abs_debt_per_diff,debt,equity,firm_value,leverage,ROE,V0,r,alpha,pi,xi
,Float32,Float32,Float32,Float32,Float32,Float32,Float32,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float32,Float32,Float32,Float32,Float32
1,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,2.0,20.2187,22.5321,0.0383904,5.0437e-6,2.73432e-6,0.00164775,true,6.09102e-7,0.00189876,0.00189876,20.2571,82.9778,103.235,0.196223,1.0,100.0,0.08,0.6,0.27,1.0
2,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,2.0,20.5511,22.9242,0.02309,-7.07755e-6,4.66086e-6,0.00169912,true,-2.23847e-6,0.00112354,0.00112354,20.5742,82.5915,103.166,0.199428,1.0,100.0,0.08,0.6,0.27,1.0
3,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,2.0,20.8834,23.3163,0.00778956,-1.68925e-5,-1.8649e-5,0.00175788,true,6.01445e-7,0.000373002,0.000373002,20.8912,82.2049,103.096,0.202638,1.0,100.0,0.08,0.6,0.27,1.0
4,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,2.0,21.2158,23.7087,0.00751087,1.66272e-5,1.50645e-5,0.00182392,true,6.01011e-7,-0.000354023,0.000354023,21.2083,81.8174,103.026,0.205854,1.0,100.0,0.08,0.6,0.27,1.0
5,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,2.0,21.5481,24.1007,0.0228113,-1.76675e-5,-4.03347e-6,0.00188559,true,1.08495e-6,-0.00105862,0.00105862,21.5253,81.4308,102.956,0.209073,1.0,100.0,0.08,0.6,0.27,1.0
6,0.02,0.02,0.0,0.015,0.3,0.2,0.2,1.0,1.0,2.0,21.8805,24.4929,0.0381117,-8.72211e-6,2.08788e-6,0.00193996,true,-3.62949e-6,-0.00174181,0.00174181,21.8424,81.0433,102.886,0.212298,1.0,100.0,0.08,0.6,0.27,1.0


In [125]:
df

,c,p,vb,debt,abs_debt_diff,debt_per_diff,abs_debt_per_diff,count,time
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,UInt64
1,2.0,17.8922,14.2248,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016acc38
2,2.0,17.8922,15.5764,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016acc38
3,2.0,17.8922,16.928,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016acc38
4,2.0,17.8922,18.2796,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016acc38
5,2.0,17.8922,19.6312,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016acc38
6,2.0,17.8922,20.9827,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016acc38
7,2.0,17.8922,22.3343,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016acc38
8,2.0,17.8922,23.6859,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016acc38
9,2.0,17.8922,25.0375,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016acc38


In [ ]:
        main_params = [:gross_delta, :delta, :kappa, :lambda, :sigmah, :sigmal]
        k_struct_params = [:mu_b, :m, :c, :p, :vb]
        fixed_params = [:V0, :r, :alpha , :pi, :xi]
        result_vars = [:abs_debt_diff, 
                   :eq_deriv,       
                   :eq_deriv_min_val, 
                   :eq_min_val,    
                   :eq_negative,     
                   :eq_vb, 
                   :debt_per_diff,
                   :abs_debt_per_diff]
        share_values = [:debt, :equity, :firm_value, :leverage, :ROE]

In [85]:
tmp1 = fetch(@spawn [eq_deriv_root_search(svm, eqdf_all, p; params_dict=params_dict) 
                    for p in unique(eqdf_all[:p])])

RemoteException(1, CapturedException(MethodError(getfield(Main, Symbol("#kw##eq_deriv_root_search"))(), ((params_dict = Dict{Any,Any}(:alpha=>0.6,:c=>2.0,:delta=>0.02,:pi=>0.27,:lambda=>0.3,:gross_delta=>0.02,:V0=>100.0,:kappa=>0.015,:sigmal=>0.2,:iota=>0.0…),), eq_deriv_root_search, Main.ModelObj.Firm
  mu_b: Float64 1.0
  m: Float64 1.0
  c: Float64 2.0
  p: Float64 NaN
  vbl: Float64 NaN
  vbh: Float64 NaN
  pm: Main.ModelObj.FirmParams
  bi: Main.ModelObj.BPrInputs
  bs: Main.ModelObj.BPrSurfs
  bf: Main.ModelObj.BPrInterpFuns
, 96×30 DataFrame. Omitted printing of 24 columns
│ Row │ gross_delta │ delta   │ iota    │ kappa   │ lambda  │ sigmah  │
│     │ Float32     │ Float32 │ Float32 │ Float32 │ Float32 │ Float32 │
├─────┼─────────────┼─────────┼─────────┼─────────┼─────────┼─────────┤
│ 1   │ 0.02        │ 0.02    │ 0.0     │ 0.015   │ 0.3     │ 0.2     │
│ 2   │ 0.02        │ 0.02    │ 0.0     │ 0.015   │ 0.3     │ 0.2     │
│ 3   │ 0.02        │ 0.02    │ 0.0     │ 0.015   │ 0

In [254]:
p = unique(eqdf_all[:p])[3]
N = 10000

tmp = eqdf_all[abs.(eqdf_all[:p] .- p) .< 1e-4, :]

eq_deriv_fun = Dierckx.Spline1D(tmp[:vb], tmp[:eq_deriv], k=3, bc="extrapolate")
debt_fun = Dierckx.Spline1D(tmp[:vb], tmp[:debt], k=3, bc="extrapolate")

ref_vbgrid = range(minimum(tmp[:vb]), stop=maximum(tmp[:vb]), length=N)

vbOpt = ref_vbgrid[argmin(abs.(eq_deriv_fun(ref_vbgrid)))]
eq_derivOpt = eq_deriv_fun(vbOpt)
debtOpt = debt_fun(vbOpt)
abs_debt_diff = debtOpt - AnalyticFunctions.get_agg_p(svm, p=p)


0.007789560159125131

In [256]:
abs_debt_diff

0.007789560159125131

In [247]:
eq_deriv_fun[ref_vbgrid]

MethodError: MethodError: no method matching getindex(::Spline1D, ::StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}})

In [230]:
EqFinDiff.eq_fd(svm, vb; mu_b=svm.mu_b, c=svm.c, p=unique(df[:p]))

No documentation found.

`Main.EqFinDiff.eq_fd` is a `Function`.

```
# 1 method for generic function "eq_fd":
[1] eq_fd(svm, vbl; mu_b, c, p, debt) in Main.EqFinDiff at /home/artur/BondPricing/Julia/modules/EqFinDiff/eq_fin_diff_funs.jl:238
```


In [132]:
function eq_deriv_root_search(self, pOpt, vbOpt)
    count = 0
    eq_deriv = 1
    eqdf_all = nothing

    println(string("pOpt: ", pOpt))
    println(string("vbOpt: ", vbOpt))

    while (count < 5) & (abs(eq_deriv) > .005)
        println("=================================== ")
        println(string("Count for (c, p) = (", svm.c, 
                       ", ", pOpt, "): ", count))
        print(string("Count: ", count))
        print(" ")
        eq_df_all = self.search_eq_deriv(pOpt, vbOpt, df=eq_df_all)
        vbOpt = self.interp_deriv(eq_df_all)
        derivloc = eq_df_all['eq_deriv'] ** 2 == np.min(eq_df_all['eq_deriv'] ** 2)
        eq_deriv = eq_df_all.loc[derivloc, 'eq_deriv_min_val'].values[0]
        print('Equity Derivative: ' + str(eq_deriv))
        print('===================================')
        count += 1
        end

    # Computing Debt At Par
    print(' ')
    print('Computing Debt at Par Variables')
    eq_df_all.reset_index(drop=True, inplace=True)
    eq_df_all['debt_diff'] = (eq_df_all['debt'].values - eq_df_all['p'].values * self.svm.get_m())
    eq_df_all['debt_per_diff'] = eq_df_all['debt_diff'].values/(eq_df_all['p'].values * self.svm.get_m())
    print(' ')

    # time.sleep(10)
    print(' ========================')
    print('Updating queue for (c, p) = (' +
              str(self.svm.get_c()) + ', ' + str(pOpt) + ')...')
    if manager:
        mq.update({pOpt:  eq_df_all})
    else:
        # Add results to queue:
        mq.put({pOpt: eq_df_all})

    print('Updated!')
    print(' ========================')



,c,p,vb,debt,abs_diff,abs_per_diff,per_diff,count,time
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,UInt64
1,2.0,17.8922,14.2248,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f
2,2.0,17.8922,15.5764,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f
3,2.0,17.8922,16.928,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f
4,2.0,17.8922,18.2796,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f
5,2.0,17.8922,19.6312,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f
6,2.0,17.8922,20.9827,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f
7,2.0,17.8922,22.3343,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f
8,2.0,17.8922,23.6859,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f
9,2.0,17.8922,25.0375,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f


In [129]:
df3 = conditionally_load_df(:df4, debt_at_par_cp_path_fname)

,c,p,vb,debt,abs_diff,abs_per_diff,per_diff,count,time
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,UInt64
1,2.0,17.8922,14.2248,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f
2,2.0,17.8922,15.5764,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f
3,2.0,17.8922,16.928,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f
4,2.0,17.8922,18.2796,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f
5,2.0,17.8922,19.6312,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f
6,2.0,17.8922,20.9827,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f
7,2.0,17.8922,22.3343,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f
8,2.0,17.8922,23.6859,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f
9,2.0,17.8922,25.0375,18.0377,0.145493,0.00813169,0.00813169,0,0x00000000016d8e0f


In [107]:
isdefined("main", DFI)

TypeError: TypeError: in isdefined, expected Symbol, got DataFrame

In [96]:
function check_if_var_is_defined(x)
           isdefined(:x)       
end

check_if_var_is_defined (generic function with 1 method)

In [102]:
isdefined()

UndefVarError: UndefVarError: dfi not defined

In [101]:
check_if_var_is_defined(DFI)

ArgumentError: ArgumentError: isdefined: too few arguments (expected 2)

In [17]:
println(string("Part I DF length: ", size(DFI, 1)))

Part I DF length: 320


In [93]:
df2 = Batch.filter_batch_I_df(DFI)

,c,p,vb,debt,abs_diff,abs_per_diff,per_diff,count,time
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,UInt64
1,2.0,20.2187,16.2011,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19
2,2.0,20.2187,17.7066,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19
3,2.0,20.2187,19.2121,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19
4,2.0,20.2187,20.7177,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19
5,2.0,20.2187,22.2232,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19
6,2.0,20.2187,23.7287,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19
7,2.0,20.2187,25.2342,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19
8,2.0,20.2187,26.7397,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19
9,2.0,20.2187,28.2452,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19


In [55]:
df2[60:80, :]

,c,p,vb,debt,abs_diff,abs_per_diff,per_diff,count,time
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,UInt64
1,2.0,21.5481,33.0021,21.5253,0.0228113,0.00105862,-0.00105862,0,0x00000000016c6cc1
2,2.0,21.5481,33.2609,21.5253,0.0228113,0.00105862,-0.00105862,0,0x00000000016c6cc1
3,2.0,21.5481,34.4275,21.5253,0.0228113,0.00105862,-0.00105862,0,0x00000000016c6cc1
4,2.0,21.5481,35.8529,21.5253,0.0228113,0.00105862,-0.00105862,0,0x00000000016c6cc1
5,2.0,21.5481,37.2783,21.5253,0.0228113,0.00105862,-0.00105862,0,0x00000000016c6cc1
6,2.0,21.8805,17.5899,21.8424,0.0381117,0.00174181,-0.00174181,0,0x00000000016b9a03
7,2.0,21.8805,18.9962,21.8424,0.0381117,0.00174181,-0.00174181,0,0x00000000016b9a03
8,2.0,21.8805,20.4025,21.8424,0.0381117,0.00174181,-0.00174181,0,0x00000000016b9a03
9,2.0,21.8805,21.8088,21.8424,0.0381117,0.00174181,-0.00174181,0,0x00000000016b9a03


In [34]:
pgrid = [p for p in unique(DFI[:p]) if abs_debt_diff_cond(DFI, p)]

13-element Array{Float64,1}:
 20.218699501529102
 20.551061685115883
 20.88342386870266 
 21.21578605228944 
 21.54814823587622 
 21.880510419463   
 22.21287260304978 
 22.54523478663656 
 22.87759697022334 
 23.209959153810118
 23.5423213373969  
 23.874683520983677
 24.207045704570458

In [41]:
[(p in pgrid) for p in DFI[:, :p]]

320-element Array{Bool,1}:
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
 false
     ⋮
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true
  true

In [42]:
DFI[[(p in pgrid) for p in DFI[:, :p]], : ]

,c,p,vb,debt,abs_diff,abs_per_diff,per_diff,count,time
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,UInt64
1,2.0,20.2187,16.2011,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19
2,2.0,20.2187,17.7066,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19
3,2.0,20.2187,19.2121,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19
4,2.0,20.2187,20.7177,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19
5,2.0,20.2187,22.2232,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19
6,2.0,20.2187,23.7287,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19
7,2.0,20.2187,25.2342,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19
8,2.0,20.2187,26.7397,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19
9,2.0,20.2187,28.2452,20.2571,0.0383904,0.00189876,0.00189876,0,0x0000000001707f19


In [295]:
LL = @time fetch(@spawn [Batch.svm_debt_at_par(svm, df[1, :vblmin], df[1, :vblmax], 2.; p=p) 
    for p in df[:p]])

vb root: 25.00776752697503
Condition 1: true
Condition 2: true
Condition 3: true
Conds: true
Counter: 0
Unique vb values: 16
Debt at par: preparing results...
returning results...
length: 16
vb root: 29.86860688065107
Condition 1: true
Condition 2: true
Condition 3: true
Conds: true
Counter: 0
Unique vb values: 16
Debt at par: preparing results...
returning results...
length: 16
vb root: 21.79817913789524
Condition 1: true
Condition 2: true
Condition 3: true
Conds: true
Counter: 0
Unique vb values: 16
Debt at par: preparing results...
returning results...
length: 16
vb root: 33.31241264016109
Condition 1: true
Condition 2: true
Condition 3: true
Conds: true
Counter: 0
Unique vb values: 16
Debt at par: preparing results...
returning results...
length: 16
vb root: 32.3496394128509
Condition 1: true
Condition 2: true
Condition 3: true
Conds: true
Counter: 0
Unique vb values: 16
Debt at par: preparing results...
returning results...
length: 16
vb root: 32.70942014229658
Condition 1: true
C

15-element Array{DataFrame,1}:
 16×9 DataFrame. Omitted printing of 3 columns
│ Row │ c       │ p       │ vb      │ debt    │ abs_diff │ abs_per_diff │
│     │ Float64 │ Float64 │ Float64 │ Float64 │ Float64  │ Float64      │
├─────┼─────────┼─────────┼─────────┼─────────┼──────────┼──────────────┤
│ 1   │ 2.0     │ 17.8922 │ 14.2248 │ 18.0377 │ 0.145493 │ 0.00813169   │
│ 2   │ 2.0     │ 17.8922 │ 15.5764 │ 18.0377 │ 0.145493 │ 0.00813169   │
│ 3   │ 2.0     │ 17.8922 │ 16.928  │ 18.0377 │ 0.145493 │ 0.00813169   │
│ 4   │ 2.0     │ 17.8922 │ 18.2796 │ 18.0377 │ 0.145493 │ 0.00813169   │
│ 5   │ 2.0     │ 17.8922 │ 19.6312 │ 18.0377 │ 0.145493 │ 0.00813169   │
│ 6   │ 2.0     │ 17.8922 │ 20.9827 │ 18.0377 │ 0.145493 │ 0.00813169   │
│ 7   │ 2.0     │ 17.8922 │ 22.3343 │ 18.0377 │ 0.145493 │ 0.00813169   │
│ 8   │ 2.0     │ 17.8922 │ 23.6859 │ 18.0377 │ 0.145493 │ 0.00813169   │
│ 9   │ 2.0     │ 17.8922 │ 25.0078 │ 18.0377 │ 0.145493 │ 0.00813169   │
│ 10  │ 2.0     │ 17.8922 │ 25.037

In [291]:
Batch.svm_debt_at_par(svm, df[1, :vblmin], df[1, :vblmax], 2.; p=df[1, :p])

vb root: 25.00776752697503
16×5 DataFrame
│ Row │ vb      │ debt    │ abs_diff │ per_diff   │ abs_per_diff │
│     │ Float64 │ Float64 │ Float64  │ Float64    │ Float64      │
├─────┼─────────┼─────────┼──────────┼────────────┼──────────────┤
│ 1   │ 14.2248 │ 18.0377 │ 0.145493 │ 0.00813169 │ 0.00813169   │
│ 2   │ 15.5764 │ 18.0377 │ 0.145493 │ 0.00813169 │ 0.00813169   │
│ 3   │ 16.928  │ 18.0377 │ 0.145493 │ 0.00813169 │ 0.00813169   │
│ 4   │ 18.2796 │ 18.0377 │ 0.145493 │ 0.00813169 │ 0.00813169   │
│ 5   │ 19.6312 │ 18.0377 │ 0.145493 │ 0.00813169 │ 0.00813169   │
│ 6   │ 20.9827 │ 18.0377 │ 0.145493 │ 0.00813169 │ 0.00813169   │
│ 7   │ 22.3343 │ 18.0377 │ 0.145493 │ 0.00813169 │ 0.00813169   │
│ 8   │ 23.6859 │ 18.0377 │ 0.145493 │ 0.00813169 │ 0.00813169   │
│ 9   │ 25.0078 │ 18.0377 │ 0.145493 │ 0.00813169 │ 0.00813169   │
│ 10  │ 25.0375 │ 18.0377 │ 0.145493 │ 0.00813169 │ 0.00813169   │
│ 11  │ 26.3891 │ 18.0377 │ 0.145493 │ 0.00813169 │ 0.00813169   │
│ 12  │ 27.7407 │ 18

,c,p,vb,debt,abs_diff,abs_per_diff,per_diff,count,time
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Int64,UInt64
1,2.0,17.8922,14.2248,18.0377,0.145493,0.00813169,0.00813169,0,0x000000001da4e32a
2,2.0,17.8922,15.5764,18.0377,0.145493,0.00813169,0.00813169,0,0x000000001da4e32a
3,2.0,17.8922,16.928,18.0377,0.145493,0.00813169,0.00813169,0,0x000000001da4e32a
4,2.0,17.8922,18.2796,18.0377,0.145493,0.00813169,0.00813169,0,0x000000001da4e32a
5,2.0,17.8922,19.6312,18.0377,0.145493,0.00813169,0.00813169,0,0x000000001da4e32a
6,2.0,17.8922,20.9827,18.0377,0.145493,0.00813169,0.00813169,0,0x000000001da4e32a
7,2.0,17.8922,22.3343,18.0377,0.145493,0.00813169,0.00813169,0,0x000000001da4e32a
8,2.0,17.8922,23.6859,18.0377,0.145493,0.00813169,0.00813169,0,0x000000001da4e32a
9,2.0,17.8922,25.0078,18.0377,0.145493,0.00813169,0.00813169,0,0x000000001da4e32a


In [289]:
AnalyticFunctions.rf_debt(svm.mu_b, 1., 2., 17.8922, .08, 1., svm.pm.kappa)

18.037677478565442

In [224]:
x = [1, 2, 3, 4, 5]
u = [0.00813169, 4, 0.00813169, 0.00813169, 8]
reverse(x)[argmin(abs.(reverse(u)))]

4

In [217]:
?argmin()

```
argmin(itr) -> Integer
```

Return the index of the minimum element in a collection. If there are multiple minimal elements, then the first one will be returned.

The collection must not be empty.

# Examples

```jldoctest
julia> argmin([8,0.1,-9,pi])
3

julia> argmin([7,1,1,6])
2

julia> argmin([7,1,1,NaN])
4
```

---

```
argmin(A; dims) -> indices
```

For an array input, return the indices of the minimum elements over the given dimensions. `NaN` is treated as less than all other values.

# Examples

```jldoctest
julia> A = [1.0 2; 3 4]
2×2 Array{Float64,2}:
 1.0  2.0
 3.0  4.0

julia> argmin(A, dims=1)
1×2 Array{CartesianIndex{2},2}:
 CartesianIndex(1, 1)  CartesianIndex(1, 2)

julia> argmin(A, dims=2)
2×1 Array{CartesianIndex{2},2}:
 CartesianIndex(1, 1)
 CartesianIndex(2, 1)
```


In [140]:
?Batch.svm_debt_at_par()

No documentation found.

`Main.Batch.svm_debt_at_par` is a `Function`.

```
# 1 method for generic function "svm_debt_at_par":
[1] svm_debt_at_par(svm, x_min, x_max, c; p, vbl, f_low, f_high, disp_option, precise) in Main.Batch at /home/artur/BondPricing/Julia/modules/Batch/_batch_debt_at_par_funs.jl:138
```


In [102]:
size(collect(LL[:,:]))

(20, 1)

In [92]:
DataFrame(LL[:,:], [:p, :vblmin, :vblmax])

DimensionMismatch: DimensionMismatch("Number of columns (20) and number of column names (3) are not equal")

In [34]:
[i for i in Batch.bounded_vbs(svm, count, p)]

2-element Array{Float64,1}:
 0.789794795582146 
 1.6584861381223406

In [124]:
LL = [vcat(p, [i for i in Batch.bounded_vbs(svm, count, p)]) for p in count["adj_pgrid"]]
DataFrame(transpose(hcat(LL...)), [:p, :vblmin, :vblhmax])

20-element Array{Array{Float64,1},1}:
 [17.8922, 14.2162, 29.8525]
 [18.2245, 14.497, 30.4421] 
 [18.5569, 14.7778, 31.0318]
 [18.8893, 15.0586, 31.6214]
 [19.2216, 15.3394, 32.2111]
 [19.554, 15.6202, 32.8007] 
 [19.8863, 15.901, 33.3903] 
 [20.2187, 16.1817, 33.98]  
 [20.5511, 16.4625, 34.5696]
 [20.8834, 16.7433, 35.1592]
 [21.2158, 17.0241, 35.7489]
 [21.5481, 17.3049, 36.3385]
 [21.8805, 17.5857, 36.9282]
 [22.2129, 17.8665, 37.5178]
 [22.5452, 18.1473, 38.1074]
 [22.8776, 18.4281, 38.6971]
 [23.21, 18.7089, 39.2867]  
 [23.5423, 18.9897, 39.8764]
 [23.8747, 19.2705, 40.466] 
 [24.207, 19.5513, 41.0556] 

,p,vblmin,vblhmax
,Float64,Float64,Float64
1,17.8922,14.2162,29.8525
2,18.2245,14.497,30.4421
3,18.5569,14.7778,31.0318
4,18.8893,15.0586,31.6214
5,19.2216,15.3394,32.2111
6,19.554,15.6202,32.8007
7,19.8863,15.901,33.3903
8,20.2187,16.1817,33.98
9,20.5511,16.4625,34.5696


In [113]:
DataFrame(Array(LL[:, :]), [:a])

,a
,Tuple…
1,"(14.2162, 29.8525)"
2,"(14.497, 30.4421)"
3,"(14.7778, 31.0318)"
4,"(15.0586, 31.6214)"
5,"(15.3394, 32.2111)"
6,"(15.6202, 32.8007)"
7,"(15.901, 33.3903)"
8,"(16.1817, 33.98)"
9,"(16.4625, 34.5696)"


In [86]:
Matrix(LL, (20, 3))

MethodError: MethodError: no method matching Array{T,2} where T(::Array{Array{Float64,1},1}, ::Tuple{Int64,Int64})
Closest candidates are:
  Array{T,2} where T(::AbstractArray{S,N}) where {S, N} at boot.jl:424
  Array{T,2} where T(!Matched::LinearAlgebra.UniformScaling, ::Tuple{Int64,Int64}) at /buildworker/worker/package_linux64/build/usr/share/julia/stdlib/v1.1/LinearAlgebra/src/uniformscaling.jl:342

In [83]:
DataFrame(p =[v[1] for v in (LL)], vblmin = [v[2] for v in (LL)], vblmax = [v[3] for v in (LL)])

,p,vblmin,vblmax
,Float64,Float64,Float64
1,17.8922,14.2162,29.8525
2,18.2245,14.497,30.4421
3,18.5569,14.7778,31.0318
4,18.8893,15.0586,31.6214
5,19.2216,15.3394,32.2111
6,19.554,15.6202,32.8007
7,19.8863,15.901,33.3903
8,20.2187,16.1817,33.98
9,20.5511,16.4625,34.5696


In [74]:
reshape(LL, (20, 3))

DimensionMismatch: DimensionMismatch("new dimensions (20, 3) must be consistent with array size 20")

In [40]:
using DataFrames

In [50]:
LL2 = [(LL[i][1], LL[i][2], LL[i][3]) for i in range(1, stop=length(LL))]

20-element Array{Tuple{Float64,Float64,Float64},1}:
 (17.892164216421644, 14.216188852690687, 29.852503816182185)
 (18.224526400008425, 14.496982933707425, 30.442141901464588)
 (18.556888583595203, 14.777777014724162, 31.031779986746983)
 (18.889250767181984, 15.058571095740898, 31.621418072029385)
 (19.221612950768762, 15.339365176757633, 32.21105615731177) 
 (19.553975134355543, 15.620159257774365, 32.80069424259416) 
 (19.88633731794232, 15.9009533387911, 33.39033232787656)    
 (20.218699501529102, 16.181747419807834, 33.97997041315895) 
 (20.551061685115883, 16.462541500824575, 34.569608498441355)
 (20.88342386870266, 16.743335581841304, 35.15924658372374)  
 (21.21578605228944, 17.024129662858044, 35.748884669006145) 
 (21.54814823587622, 17.304923743874777, 36.33852275428853)  
 (21.880510419463, 17.58571782489151, 36.92816083957092)     
 (22.21287260304978, 17.866511905908247, 37.51779892485332)  
 (22.54523478663656, 18.147305986924984, 38.10743701013572)  
 (22.8775969702233

In [55]:
typeof(LL2)

Array{Tuple{Float64,Float64,Float64},1}

In [45]:
LL[1]

3-element Array{Float64,1}:
 17.892164216421644
 14.216188852690687
 29.852503816182185

In [44]:
typeof(LL)

Array{Array{Float64,1},1}

In [56]:
DataFrame(LL2, cols=["p", "vblmin", "vblmax"])

MethodError: MethodError: no method matching DataFrame(::Array{Tuple{Float64,Float64,Float64},1}; cols=["p", "vblmin", "vblmax"])
Closest candidates are:
  DataFrame(::AbstractArray{T,1} where T, !Matched::AbstractArray{Symbol,1}; makeunique) at /home/artur/.julia/packages/DataFrames/lyCjP/src/dataframe/dataframe.jl:160 got unsupported keyword argument "cols"
  DataFrame(::Any) at /home/artur/.julia/packages/DataFrames/lyCjP/src/other/tables.jl:17 got unsupported keyword argument "cols"
  DataFrame(::Any, !Matched::DataStreams.Data.Schema, !Matched::Type{S}, !Matched::Bool; reference) where S at /home/artur/.julia/packages/DataFrames/lyCjP/src/abstractdataframe/io.jl:446 got unsupported keyword argument "cols"
  ...

In [402]:
bt = Batch.BatchObj()
comb_num = 2 
bt = Batch.set_par_dict(bt, comb_num)

# svm = ModelObj.firm_constructor(bt.mi._svm_dict)

bt.mi._svm_dict["lambda"] = .3
bt.mi._svm_dict["sigmal"] = .2
bt.mi._svm_dict["sigmah"] = .2
bt.mi._svm_dict["kappa"] = 0.015

# Set Directories & Paths (Main, Batch, Maturity, Combination)
bt = Batch.set_comb_res_paths(bt)

# Create Directories
bt = Batch.mk_comb_res_dirs(bt)

Main.Batch.BatchStruct
  main_dir: String "Julia"
  res_dir: String "Results/"
  mat_dir_prefix: String "m_"
  coupon_dir_prefix: String "c_"
  svm_partI_fn_prefix: String "svm_data_c"
  svm_partII_fn_prefix: String "svm_res_c"
  svm_param_values_dict: Dict{String,Array{T,1} where T}
  svm_coupon_grid: StepRangeLen{Float64,Base.TwicePrecision{Float64},Base.TwicePrecision{Float64}}
  pvb_tol_vec: Array{Float64}((4,)) [1.0e-6, 1.0e-5, 0.0001, 0.001]
  _common_params: Dict{String,Float64}
  _params_order: Array{String}((9,))
  _params_combs: Array{Array{Float64,1}}((1, 1, 1, 5, 1, 5, 1, 1, 6))
  mi: Main.Batch.ModelInputs
  coupon_grid: StepRange{Int64,Int64}


In [481]:
comb_num = 2
skip_bpr_computation = 0
cpos = 1
skip_c_input = 1
skip_sol_input = 0
sys_argv = [comb_num, skip_bpr_computation, cpos, skip_c_input, skip_sol_input]

length(sys_argv)

5

In [396]:
svm

Main.ModelObj.Firm
  mu_b: Float64 1.0
  m: Float64 1.0
  c: Float64 NaN
  p: Float64 NaN
  vbl: Float64 NaN
  vbh: Float64 NaN
  pm: Main.ModelObj.FirmParams
  bi: Main.ModelObj.BPrInputs
  bs: Main.ModelObj.BPrSurfs
  bf: Main.ModelObj.BPrInterpFuns


In [395]:
BondPrInterp.get_cvm_debt_price(svm2, vb, svm2.pm.sigmal, c=2., p=40.)

39.127749949905755

In [65]:
c = 3.
pgrid = range(10, stop=.9 * svm.pm.V0, length=100)

cvm_vbl_fun, cvm_vbh_fun = get_cvm_vb_funs(svm, c, pgrid)

(Spline1D(knots=[10.0,11.6162 … 88.3838,90.0] (98 elements), k=3, extrapolation="extrapolate", residual=0.0), Spline1D(knots=[10.0,11.6162 … 88.3838,90.0] (98 elements), k=3, extrapolation="extrapolate", residual=0.0))

In [68]:
using DataFrames

In [241]:
df = DataFrame(vblmin = Float64[], vblmax = Float64[])
for p in pgrid
    vblmin = .7 * minimum([cvm_vbl_fun(p), cvm_vbh_fun(p)])
    vblmax = 1.3 * maximum([cvm_vbl_fun(p), cvm_vbh_fun(p)])
    df = push!(df, (vblmin, vblmax))
#     df.append(bt.svm.jdebt_at_par(vblmin, vblmax, p=p))
end

In [391]:
?BondPrInterp.get_cvm_debt_price(svm, vb, svm.pm.sigmal; c=c, p=p)

No documentation found.

`Main.BondPrInterp.get_cvm_debt_price` is a `Function`.

```
# 1 method for generic function "get_cvm_debt_price":
[1] get_cvm_debt_price(svm, vb, sigma; Vt, mu_b, c, p, N1, N2) in Main.BondPrInterp at /home/artur/BondPricing/Julia/modules/BondPrInterp/debtpr_funs.jl:25
```


In [ ]:
function 

In [ ]:
# Batch.
function get_pgrid(svm, lower_adj=.85, upper_adj=1.15):
    # (i) Compute Values for Debt at Par in Baseline Model Model
    cvmh_par = self.svm._cvmh.get_debt_at_par()
    cvml_par = self.svm._cvml.get_debt_at_par()

    # (ii) ########## Form P Grid: #########
    cvm_ph = np.float(cvmh_par['P'] / self.svm.get_m())
    cvm_pl = np.float(cvml_par['P'] / self.svm.get_m())

    # Notice that RF-Debt(P) < P for all P < pmax1,
    # since (1- np.exp(- rdisc * svm.get_m())/(rdisc * svm.get_m()) < 1
    p_max = self.svm.get_C()/self.svm.get_rdisc()
    p_low = lower_adj * np.min([cvm_pl, cvm_ph])
    p_high = upper_adj * np.min([np.max([cvm_pl, cvm_ph]), p_max])
    p_grid = np.linspace(p_low, p_high, 10)

    return p_grid



In [ ]:
# #######################################################
# ##################### DEBT AT PAR #####################
# #######################################################





In [397]:
?AnalyticFunctions.get_cvm_vb()

No documentation found.

`Main.AnalyticFunctions.get_cvm_vb` is a `Function`.

```
# 1 method for generic function "get_cvm_vb":
[1] get_cvm_vb(svm, sigma; mu_b, c, p) in Main.AnalyticFunctions at /home/artur/BondPricing/Julia/modules/AnalyticFunctions/af_get_methods.jl:14
```


In [ ]:
function get_cvm_p_debt(svm, sigma, mu_b, c, p)
    vb = AnalyticFunctions.get_cvm_vb(svm, sigma; mu_b=mu_b, c=c, p=p)
    
    return BondPrInterp.get_cvm_debt_price(svm, vb, sigma; mu_b=mu_b, c=c, p=p)
end

In [400]:
a0 = [1, 2, 3]
a1 = [4, 5, 6]

a1 - a0

3-element Array{Int64,1}:
 3
 3
 3

In [406]:
function get_cvm_vb_funs(svm, mu_b, c, pgrid)
    cvm_vbl = [AnalyticFunctions.get_cvm_vb(svm, svm.pm.sigmal; 
                                            mu_b=mu_b, c=c, p=p) for p in pgrid]
    cvm_vbh = [AnalyticFunctions.get_cvm_vb(svm, svm.pm.sigmah; 
                                            mu_b=mu_b, c=c, p=p) for p in pgrid]
    cvm_vbl_fun = Dierckx.Spline1D(pgrid, cvm_vbl, k=3, bc="extrapolate")
    cvm_vbh_fun = Dierckx.Spline1D(pgrid, cvm_vbh, k=3, bc="extrapolate")

    return cvm_vbl_fun, cvm_vbh_fun
end

get_cvm_vb_funs (generic function with 2 methods)

In [444]:
function get_cvm_p_debt_diff(svm, sigma, mu_b, c, pgrid; N=10000)
    # Compute CVM VBs
    cvm_vb = [AnalyticFunctions.get_cvm_vb(svm, sigma; 
                                 mu_b=mu_b, c=c, p=p) for p in pgrid]
    
    # Interpolate CVM VBs
    cvm_vb_fun = Dierckx.Spline1D(pgrid, cvm_vb, k=3, bc="extrapolate")
    
    # Compute Debt Values
    cvm_debt = fetch(@spawn [BondPrInterp.get_cvm_debt_price(svm, 
                                            cvm_vb_fun(p), sigma; 
                                            mu_b=mu_b, c=c, p=p) for p in pgrid])
    
    # Compute Debt-P
    aggPgrid = [AnalyticFunctions.get_agg_p(svm; 
                                    mu_b=mu_b, p=p) for p in pgrid]
    cvm_debt_diff = cvm_debt - aggPgrid
    
    # Interpolate and Find Optimal Value
    pgrid_ref = range(pgrid[1], stop=pgrid[end], length=N)
    cvm_dff = Dierckx.Spline1D(pgrid, cvm_debt_diff, k=3, bc="extrapolate")
    cvm_pOpt = pgrid_ref[argmin(abs.(cvm_dff(pgrid_ref)))]
    cvm_vbOpt = cvm_vb_fun(cvm_pOpt)
    
    return Dict("cvm_vb_fun" => cvm_vb_fun,
                "cvm_debt" => cvm_debt,
                "cvm_dff" => cvm_dff,
                "cvm_pOpt" => cvm_pOpt,
                "cvm_vbOpt" => cvm_vbOpt)
end

get_cvm_p_debt_diff (generic function with 4 methods)

In [456]:
function test(svm; mu_b=NaN, c=NaN, N=10000, lower_adj=.85, upper_adj=1.15)
    if isnan(mu_b)
        mu_b = svm.mu_b
    end
    
    if isnan(c)
        c = svm.c
    end
        
    # Form p grids
    pgrid = range(10, stop=.9 * svm.pm.V0, length=15)

    # Get VB, Debt, Debt Diff, Debt at Par values
    cvml_dict = get_cvm_p_debt_diff(svm, svm.pm.sigmal, mu_b, c, pgrid; N=N)
    cvmh_dict = get_cvm_p_debt_diff(svm, svm.pm.sigmah, mu_b, c, pgrid; N=N)
    
    # Form Adjusted pgrid
    p_lower = lower_adj * minimum([cvml_dict["cvm_pOpt"], cvmh_dict["cvm_pOpt"]])
    p_upper = upper_adj * maximum([cvml_dict["cvm_pOpt"], cvmh_dict["cvm_pOpt"]])
    adj_pgrid = range(p_lower, stop=p_upper, length=20)
   
    # Ensure VB values are within bounds
    
    return Dict("cvml" => cvml_dict, "cvmh" => cvmh_dict, "adj_pgrid" => adj_pgrid)
end


test (generic function with 1 method)

In [457]:
rr = test(svm, c=2.)

Dict{String,Any} with 3 entries:
  "cvml"      => Dict{String,Any}("cvm_vb_fun"=>Spline1D(knots=[10.0,21.4286 … …
  "cvmh"      => Dict{String,Any}("cvm_vb_fun"=>Spline1D(knots=[10.0,21.4286 … …
  "adj_pgrid" => 17.89173917391739:0.332354288060385:24.206470647064705

In [479]:
# Because the interpolated bond pricing function 
# is only defined for vbl s.t. bhl_min <= cvm_vbh/vbl <= bhl_max,
# need to bound grid of vbl candidates:
function bounded_vbs(cont, p)
    
    cvm_vbl = cont["cvml"]["cvm_vb_fun"](p)
    cvm_vbh = cont["cvmh"]["cvm_vb_fun"](p)
    
    # Form VB grid
    vblmin = .5 * minimum([cvm_vbl, cvm_vbh])
    vblmax = 1.5 * maximum([cvm_vbl, cvm_vbh])
    vblgrid = range(vblmin, stop=vblmax, length=20)
    
    # Compute vbh/vbl ratios
    ratio = [cvm_vbh/vbl for vbl in vblgrid]

    # Interpolate ratios
    ratiof = Dierckx.Spline1D(vblgrid, ratio, k=3, bc="extrapolate")

    # Refine vblgrid
    vblgrid_ref = range(vblmin, stop=vblmax, length=10000)
    
    # Ensure vbl values satisfy bhl_min <= cvm_vbh/vbl <= bhl_max
    bounded_vblgrid = [vb for vb in vblgrid_ref if 
                        (ratiof(vb) <= svm.bi.vbhlmax) & (ratiof(vb) >= svm.bi.vbhlmin)]
    vblmin = minimum(bounded_vblgrid)
    vblmax = maximum(bounded_vblgrid)
                
    return vblmin, vblmax
end
            

bounded_vbs (generic function with 1 method)

In [486]:
cont = Batch.p_candidates(svm; c=2.)

UndefVarError: UndefVarError: AnalyticFunctions not defined

In [483]:
bounded_vbs(rr, rr["adj_pgrid"][end])

(19.550790558325303, 41.05460723682773)

In [475]:
p = rr["adj_pgrid"][1]

# Form Adjusted pgrid
cvm_vbl = rr["cvml"]["cvm_vb_fun"](p)
cvm_vbh = rr["cvmh"]["cvm_vb_fun"](p)
vblmin = .5 * minimum([cvm_vbl, cvm_vbh])
vblmax = 1.5 * maximum([cvm_vbl, cvm_vbh])
vblgrid = range(vblmin, stop=vblmax, length=20)
vblgrid_ref = range(vblmin, stop=vblmax, length=10000)
ratio = [cvm_vbh/vbl for vbl in vblgrid]

ratiof = Dierckx.Spline1D(vblgrid, ratio, k=3, bc="extrapolate")

bounded_vblgrid = [vb for vb in vblgrid_ref if 
                    (ratiof(vb) <= svm.bi.vbhlmax) & (ratiof(vb) >= svm.bi.vbhlmin)]
vblmin = minimum(bounded_vblgrid)
vblmax = maximum(bounded_vblgrid)




29.85174975551961

In [478]:
cvm_vbh

19.90116650367974

In [476]:
vblmin

14.215829758229077

In [477]:
vblmax

29.85174975551961

In [474]:
[vb for vb in vblgrid_ref if (ratiof(vb) <= svm.bi.vbhlmax) & (ratiof(vb) >= svm.bi.vbhlmin)]

7857-element Array{Float64,1}:
 14.215829758229077
 14.217820073911014
 14.219810389592949
 14.221800705274886
 14.223791020956822
 14.225781336638757
 14.227771652320694
 14.22976196800263 
 14.231752283684566
 14.233742599366503
 14.23573291504844 
 14.237723230730374
 14.239713546412311
  ⋮                
 29.829856283018312
 29.83184659870025 
 29.833836914382182
 29.83582723006412 
 29.837817545746056
 29.839807861427992
 29.84179817710993 
 29.843788492791866
 29.8457788084738  
 29.847769124155736
 29.849759439837673
 29.85174975551961 

In [465]:
vblgrid[1]

19.90116650367974

In [466]:
vblgrid[end]

19.90116650367974

In [472]:
svm.bi.vbhlmin

0.6

In [471]:
svm.bi.vbhlmax

1.4

In [453]:
df = pd.DataFrame([])
                for p in p_grid:
                    vblmin = .85 * np.min([cvm_vbl_fun(p), cvm_vbh_fun(p)])
                    vblmax = 1.15 * np.max([cvm_vbl_fun(p), cvm_vbh_fun(p)])
                    df = df.append(bt.svm.jdebt_at_par(vblmin, vblmax, p=p))



MethodError: MethodError: objects of type Int64 are not callable

In [ ]:
def get_pgrid(self, lower_adj=.85, upper_adj=1.15):
    # (i) Compute Values for Debt at Par in Baseline Model Model
    cvmh_par = self.svm._cvmh.get_debt_at_par()
    cvml_par = self.svm._cvml.get_debt_at_par()

    # (ii) ########## Form P Grid: #########
    cvm_ph = np.float(cvmh_par['P'] / self.svm.get_m())
    cvm_pl = np.float(cvml_par['P'] / self.svm.get_m())

    # Notice that RF-Debt(P) < P for all P < pmax1,
    # since (1- np.exp(- rdisc * svm.get_m())/(rdisc * svm.get_m()) < 1
    p_max = self.svm.get_C()/self.svm.get_rdisc()
    p_low = lower_adj * np.min([cvm_pl, cvm_ph])
    p_high = upper_adj * np.min([np.max([cvm_pl, cvm_ph]), p_max])
    p_grid = np.linspace(p_low, p_high, 10)

    return p_grid



In [455]:
rr = test(svm, c=2.)

Dict{String,Any} with 3 entries:
  "cvml"      => Dict{String,Any}("cvm_vb_fun"=>Spline1D(knots=[10.0,21.4286 … …
  "cvmh"      => Dict{String,Any}("cvm_vb_fun"=>Spline1D(knots=[10.0,21.4286 … …
  "adj_pgrid" => 17.89173917391739:0.332354288060385:24.206470647064705

In [447]:
rr["cvml"]

Dict{String,Any} with 5 entries:
  "cvm_vb_fun" => Spline1D(knots=[10.0,21.4286 … 78.5714,90.0] (13 elements), k…
  "cvm_dff"    => Spline1D(knots=[10.0,21.4286 … 78.5714,90.0] (13 elements), k…
  "cvm_vbOpt"  => 23.6355
  "cvm_debt"   => [10.5088, 15.96, 21.4113, 26.8625, 32.3137, 37.7649, 43.2155,…
  "cvm_pOpt"   => 21.0491

In [448]:
rr["cvmh"]

Dict{String,Any} with 5 entries:
  "cvm_vb_fun" => Spline1D(knots=[10.0,21.4286 … 78.5714,90.0] (13 elements), k…
  "cvm_dff"    => Spline1D(knots=[10.0,21.4286 … 78.5714,90.0] (13 elements), k…
  "cvm_vbOpt"  => 23.6355
  "cvm_debt"   => [10.5088, 15.96, 21.4113, 26.8625, 32.3137, 37.7649, 43.2155,…
  "cvm_pOpt"   => 21.0491

In [434]:
dd = get_cvm_p_debt_diff(svm, c=2.)

Dict{String,Any} with 10 entries:
  "cvmh_vb_fun" => Spline1D(knots=[10.0,21.4286 … 78.5714,90.0] (13 elements), …
  "cvmh_vbOpt"  => 23.6355
  "cvmh_debt"   => [10.5088, 15.96, 21.4113, 26.8625, 32.3137, 37.7649, 43.2155…
  "cvmh_pOpt"   => 21.0491
  "cvml_vb_fun" => Spline1D(knots=[10.0,21.4286 … 78.5714,90.0] (13 elements), …
  "cvml_debt"   => [10.5088, 15.96, 21.4113, 26.8625, 32.3137, 37.7649, 43.2155…
  "cvml_dff"    => Spline1D(knots=[10.0,21.4286 … 78.5714,90.0] (13 elements), …
  "cvml_pOpt"   => 21.0491
  "cvml_vbOpt"  => 23.6355
  "cvmh_dff"    => Spline1D(knots=[10.0,21.4286 … 78.5714,90.0] (13 elements), …

In [435]:
BondPrInterp.get_cvm_debt_price(svm, dd["cvml_vbOpt"], 
                        svm.pm.sigmal; c=2., p=dd["cvml_pOpt"]) - dd["cvml_pOpt"]

0.0001623638247423287

In [436]:
AnalyticFunctions.get_cvm_vb(svm, svm.pm.sigmal; c=2., p=dd["cvml_pOpt"]) - dd["cvml_vbOpt"]

7.105427357601002e-15

In [422]:
dd["cvml_vb_fun"](dd["cvml_pOpt"])

23.635452330782442

In [418]:
dd["cvml_pOpt"]

21.04910491049105

In [399]:
?BondPrInterp.get_cvm_debt_price(svm, vb, sigma; mu_b=mu_b, c=c, p=p)

No documentation found.

`Main.BondPrInterp.get_cvm_debt_price` is a `Function`.

```
# 1 method for generic function "get_cvm_debt_price":
[1] get_cvm_debt_price(svm, vb, sigma; Vt, mu_b, c, p, N1, N2) in Main.BondPrInterp at /home/artur/BondPricing/Julia/modules/BondPrInterp/debtpr_funs.jl:25
```


In [608]:
get_cvm_p_debt_diff(svm2, sigma, c, p)

RemoteException(1, CapturedException(UndefVarError(:sigma), Any[((::##316#318)(::Float64) at none:0, 1), (iterate at generator.jl:47 [inlined], 1), (collect(::Generator{StepRangeLen{Float64,TwicePrecision{Float64},TwicePrecision{Float64}},##316#318}) at array.jl:606, 1), ((::##315#317)() at macros.jl:43, 1), (run_work_thunk(::##315#317, ::Bool) at process_messages.jl:56, 1), (run_work_thunk(::RemoteValue, ::Function) at process_messages.jl:65, 1), ((::##102#104{RemoteValue,##315#317})() at task.jl:259, 1)]))

In [612]:
pgrid = range(10, stop=.9 * svm.pm.V0, length=15)
pgrid_ref = range(10, stop=.9 * svm.pm.V0, length=10000)

# p = pgrid[7]
# p_array = Float64[]
# vb_array = Float64[]
# debt_array = Float64[]
# debt_diff_array = Float64[]

c = 2.
debt_diff = fetch(@spawn [get_cvm_p_debt_diff(svm2, svm.pm.sigmal, c, p) for p in pgrid])

# Interpolate and Find Optimal Value
dff = Dierckx.Spline1D(pgrid, debt_diff)
pOpt = pgrid_ref[argmin(abs.(df(pgrid_ref)))]

# for p in pgrid
#     # Compute Results
#     vb = AnalyticFunctions.get_cvm_vb(svm, svm.pm.sigmal; c=c, p=p)
#     debt = BondPrInterp.get_cvm_debt_price(svm2, vb, svm2.pm.sigmal, c=c, p=p)
#     debt_diff = debt - AnalyticFunctions.get_agg_p(svm; p=p)
    
#     # Store Results
#     append!(p_array, p)
# #     append!(vb_array, vb)
# #     append!(debt_array, debt)
#     append!(debt_diff_array, debt_diff)
# end


# vbOpt = AnalyticFunctions.get_cvm_vb(svm, svm.pm.sigmal; c=c, p=pOpt)
# debtOpt = BondPrInterp.get_cvm_debt_price(svm2, vbOpt, svm2.pm.sigmal, c=c, p=pOpt)



21.04910491049105

In [ ]:
df(pOpt)

In [594]:
using Dierckx

df = Dierckx.Spline1D(p_array, debt_diff_array)
df(pgrid_ref[argmin(abs.(df(pgrid_ref)))])

0.0001623998210390637

In [595]:
pOpt = pgrid_ref[argmin(abs.(df(pgrid_ref)))]
vbOpt = AnalyticFunctions.get_cvm_vb(svm, svm.pm.sigmal; c=c, p=pOpt)
debtOpt = BondPrInterp.get_cvm_debt_price(svm2, vbOpt, svm2.pm.sigmal, c=c, p=pOpt)


21.049267274315792

In [597]:
pOpt

21.04910491049105

In [502]:
isfile(debt_at_par_cp_path_fname)

true

In [467]:
# nprocs => NO
comb_num = 2
skip_bpr_computation = 0
cpos = 1
skip_c_input = 0
skip_sol_input = 0
sys_arv = [comb_num, skip_bpr_computation, cpos, skip_c_input, skip_sol_input]



# Coupon Files
debt_at_par_cp_fname = string("debt_at_par_", bt.coupon_dir_prefix,
                              @sprintf("%.2f", svm.c), ".jld")
debt_at_par_cp_path_fname = joinpath(bt.mi.comb_res_path,  debt_at_par_cp_fname)
eq_fd_cp_fname = string("eq_fd_", bt.coupon_dir_prefix, @sprintf("%.2f", svm.c), ".jld")
eq_fd_cp_path_fname = joinpath(bt.mi.comb_res_path,  eq_fd_cp_fname)


# Check if results have been computed:
c_sol_files = [file for file in os.listdir(results_sub_folder) 
                    if svm_partI_fn_prefix in file]
c_res_files = [file for file in os.listdir(results_sub_folder) 
                    if svm_partII_fn_prefix in file]

# If coupon folder does not contain final results or
# if the coupon skip condition is false,
# run the calculations below:
if !isfile(eq_fd_cp_path_fname) | (skip_c is false)
    # If first part solution is missing or if skip
    # condition for the 1st part of the calculations
    # is false, run the 1st part:
    if !isfile(debt_at_par_cp_path_fname) | (skip_sol is False)
        # ############ Form VB and p grids: ############
                                            
                                            
                                
                                 
                                    end     
                                
    end                            
end
                            
                                            
                                            


┌ Warning: type Interpolations.ScaledInterpolation{Core.Float64,2,Interpolations.BSplineInterpolation{Core.Float64,2,OffsetArrays.OffsetArray{Core.Float64,2,Core.Array{Core.Float64,2}},Interpolations.BSpline{Interpolations.Cubic{Interpolations.Line{Interpolations.OnGrid}}},Core.Tuple{Base.OneTo{Core.Int64},Base.OneTo{Core.Int64}}},Interpolations.BSpline{Interpolations.Cubic{Interpolations.Line{Interpolations.OnGrid}}},Core.Tuple{Base.StepRangeLen{Core.Float64,Base.TwicePrecision{Core.Float64},Base.TwicePrecision{Core.Float64}},Base.StepRangeLen{Core.Float64,Base.TwicePrecision{Core.Float64},Base.TwicePrecision{Core.Float64}}}} not present in workspace; reconstructing
└ @ JLD /home/artur/.julia/packages/JLD/1BoSz/src/jld_types.jl:703
┌ Warning: type Interpolations.BSplineInterpolation{Core.Float64,2,OffsetArrays.OffsetArray{Core.Float64,2,Core.Array{Core.Float64,2}},Interpolations.BSpline{Interpolations.Cubic{Interpolations.Line{Interpolations.OnGrid}}},Core.Tuple{Base.OneTo{Core.Int64}

UndefVarError: UndefVarError: check_bpr_surfaces not defined

In [429]:
res = Batch.load_batch_results(bt)

┌ Warning: type Interpolations.ScaledInterpolation{Core.Float64,2,Interpolations.BSplineInterpolation{Core.Float64,2,OffsetArrays.OffsetArray{Core.Float64,2,Core.Array{Core.Float64,2}},Interpolations.BSpline{Interpolations.Cubic{Interpolations.Line{Interpolations.OnGrid}}},Core.Tuple{Base.OneTo{Core.Int64},Base.OneTo{Core.Int64}}},Interpolations.BSpline{Interpolations.Cubic{Interpolations.Line{Interpolations.OnGrid}}},Core.Tuple{Base.StepRangeLen{Core.Float64,Base.TwicePrecision{Core.Float64},Base.TwicePrecision{Core.Float64}},Base.StepRangeLen{Core.Float64,Base.TwicePrecision{Core.Float64},Base.TwicePrecision{Core.Float64}}}} not present in workspace; reconstructing
└ @ JLD /home/artur/.julia/packages/JLD/1BoSz/src/jld_types.jl:703
┌ Warning: type Interpolations.BSplineInterpolation{Core.Float64,2,OffsetArrays.OffsetArray{Core.Float64,2,Core.Array{Core.Float64,2}},Interpolations.BSpline{Interpolations.Cubic{Interpolations.Line{Interpolations.OnGrid}}},Core.Tuple{Base.OneTo{Core.Int64}

Dict{String,Any} with 2 entries:
  "bt"  => BatchStruct("Julia", "Results/", "m_", "c_", "svm_data_c", "svm_res_…
  "svm" => Firm(1.0, 1.0, NaN, NaN, NaN, NaN, FirmParams(100.0, 0.6, 0.27, 0.08…

In [457]:
["batch_obj.jld" in readdir(bt.mi.comb_res_path)]

1-element Array{Bool,1}:
 true

In [400]:
# Load Results
res = Batch.load_batch_results(bt)
svm2 = res["svm"]

# Interpolate Bond Pricing Surfaces
svm2 = @time BondPrInterp.bpr_interp(svm2)

"/mnt/ide0/home/artur/BondPricing/Julia/Results/Tests/m_1.0/xi_1.00__kappa_150_bp__gross_delta_200_bp__iota_0_bp__lambda_0.30__sigmal_0.20__sigmah_0.20"

In [212]:
modls = ["Batch", "ModelObj", "AnalyticFunctions", "BondPrInterp", "EqFinDiff"]
for modl in modls
    include(string(joinpath(module_path, modl), "/", modl, ".jl"))
end

In [213]:
svm = ModelObj.set_bpr_grids(svm)
c = 3.
p=60.
mu_b = 1.
vbl = AnalyticFunctions.get_cvm_vb(svm, svm.pm.sigmal; mu_b=mu_b, c=c, p=p)

EqFinDiff.eq_fd(svm, vbl; mu_b=mu_b, c=c, p=p)

Computing Equity Vmax
Vmax: 125.0
phi0: -59.57145870430289
phi1: 1.0
pv_rfdebt: 58.69175371123924
debt_abs_per_diff: 2.1942696727714022e-6
eq_abs_per_diff: 0.000943728074779247
Equity Vmax: 300.8274042113857
 
here
eq_max: 241.02826549810038
eq_vbl: 0.0
Computing Constant Volatility Equity Values
Finished computing Constant Volatility Equity Values
Time to compute Constant Volatility Equity Values: 0.07077312469482422
 
Shape of Gamma matrix: (1498,)
Computing equity values... 
equity: 32.590948573359285
Equity Core Function Computation Time: 4.025589942932129
Total Equity FD Core Function Computation Time: 4.45696496963501
Total computation time: 4.763500928878784


,ROE,V0,alpha,c,debt,delta,eq_deriv,eq_deriv_min_val,eq_min_val,eq_negative,eq_vb,equity,firm_value,gross_delta,iota,kappa,lambda,leverage,m,mu_b,p,pi,r,sigmah,sigmal,vb,xi
,Float64,Float32,Float32,Float64,Float64,Float32,Float64,Float64,Float64,Bool,Float64,Float64,Float64,Float32,Float32,Float32,Float32,Float64,Float64,Float64,Float64,Float32,Float32,Float32,Float32,Float64,Float32
1,-21.4951,100.0,0.6,3.0,58.4854,0.02,0.0170058,0.0170058,0.00161477,false,0.0,32.5909,91.0764,0.02,0.0,0.015,0.3,64.2158,1.0,1.0,60.0,0.27,0.08,0.2,0.2,69.0736,1.0


In [150]:
vgrid

0.8:-0.000533689126084056:0.0

In [151]:
EqFinDiff.eq_fd_newly_issued_bonds(svm, vbl, mu_b, c, p, vgrid)

ErrorException: length of x and y must match

In [211]:
BondPrInterp.get_svm_bond_price(svm, vbl, svm.m;
                                               Vt=vbl*exp((1 + 1e-4)*minimum(svm.bs.vtgrid)), mu_b=mu_b,
                                               c=c, p=p) 

33.202269636200356

In [155]:
using Distributed
using Dierckx

rfbond = AnalyticFunctions.rfbond_price(svm.m, c, p, svm.pm.r, svm.pm.xi, svm.pm.kappa)
dpayoff = AnalyticFunctions.on_default_payoff(0., vbl, svm.m,
                            mu_b, svm.m, c, p,
                            svm.pm.r, svm.pm.xi,
                            svm.pm.kappa, svm.pm.alpha)
 _, v_subgrid = ModelObj.grid_creator(minimum(svm.bs.vtgrid), maximum(vgrid), 1000)
bpr_vec = fetch(@spawn [BondPrInterp.get_svm_bond_price(svm, vbl, svm.m;
                                               Vt=vbl*exp(v), mu_b=mu_b,
                                               c=c, p=p) for v in v_subgrid])


#bpr = Dierckx.Spline1D(vcat(.0, v_subgrid), vcat(dpayoff, bpr_vec); k=3, bc="extrapolate")


RemoteException(1, CapturedException(BoundsError([0.0289191 0.029333 … 0.128093 0.128228; 0.0290263 0.0294444 … 0.129575 0.129712; … ; 0.0326117 0.0331798 … 0.24548 0.245866; 0.0326117 0.0331798 … 0.245449 0.245835], (0.0727272727272727, 1.0)), Any[(throw_boundserror(::ScaledInterpolation{Float64,2,BSplineInterpolation{Float64,2,OffsetArray{Float64,2,Array{Float64,2}},BSpline{Cubic{Line{OnGrid}}},Tuple{OneTo{Int64},OneTo{Int64}}},BSpline{Cubic{Line{OnGrid}}},Tuple{StepRangeLen{Float64,TwicePrecision{Float64},TwicePrecision{Float64}},StepRangeLen{Float64,TwicePrecision{Float64},TwicePrecision{Float64}}}}, ::Tuple{Float64,Float64}) at abstractarray.jl:484, 1), ((::ScaledInterpolation{Float64,2,BSplineInterpolation{Float64,2,OffsetArray{Float64,2,Array{Float64,2}},BSpline{Cubic{Line{OnGrid}}},Tuple{OneTo{Int64},OneTo{Int64}}},BSpline{Cubic{Line{OnGrid}}},Tuple{StepRangeLen{Float64,TwicePrecision{Float64},TwicePrecision{Float64}},StepRangeLen{Float64,TwicePrecision{Float64},TwicePrecision{

In [189]:
v = minimum(vv)
BondPrInterp.get_svm_bond_price(svm, vbl, svm.m;
                                Vt=vbl*exp(v), mu_b=mu_b,
                                c=c, p=p)

BoundsError: BoundsError: attempt to access 600×450 scale(interpolate(OffsetArray(::Array{Float64,2}, 0:601, 0:451), BSpline(Cubic(Line(OnGrid())))), (0.07272727272727274:0.0012141447867658219:0.8, 0.1111111111111111:0.001979707993071022:1.0)) with element type Float64 at index [0.0727272727272727, 1.0]

In [194]:
BondPrInterp.compute_jbpr_inputs(svm, vbl*exp(v), vbl, mu_b, c, p) 

(0.0727272727272727, 69.07363848466093, 1.0)

In [178]:
vv = reverse(ModelObj.grid_creator(.0, .8, 11)[2])

0.8:-0.07272727272727272:0.07272727272727274

In [186]:
vv

0.8:-0.07272727272727272:0.07272727272727274

In [187]:
EqFinDiff.eq_fd_newly_issued_bonds(svm, vbl, mu_b, c, p, vgrid; vtN=1000)

ErrorException: length of x and y must match

In [9]:
svm_input_path = "/home/artur/BondPricing/Python/JuliaFiles/julia_data"

# Load Bond Pricing Input Data
_params, bpr_inputd = ModelObj.load_firm_params_bpr_inputs(svm_input_path; 
                                                          file_name="_svm_inputs")

# Instantiate SVM Model Object
svm = ModelObj.firm_constructor(_params, bpr_inputs=bpr_inputd)


Setting measure of bonds to 1.0


Main.ModelObj.Firm
  mu_b: Float64 1.0
  m: Float64 1.0
  c: Float64 6.39
  p: Float64 61.68
  vbl: Float64 NaN
  vbh: Float64 68.9252769259615
  pm: Main.ModelObj.FirmParams
  bi: Main.ModelObj.BPrInputs
  bs: Main.ModelObj.BPrSurfs
  bf: Main.ModelObj.BPrInterpFuns


In [5]:
svm_input_path = "/home/artur/BondPricing/Python/JuliaFiles/julia_data"
df = CSV.read(string(svm_input_path, "/", "_svm_inputs.csv"))

# ####### Generate Inputs Dictionary #######
cols = [:vtN, :ttmN, :vbhlN, :vN, :uN]
jdict = Dict{String, Any}(string(cn) => df[1, cn] for cn in names(df) if
                          !occursin("Column", string(cn)) & !(cn in cols))
# Grid Length Numbers Should be Interger
for cn in cols
    jdict[string(cn)] = trunc(Int128, df[1, cn])
end

# Instantiate SVM Model Object
svm = ModelObj.firm_constructor(jdict)

Setting initial vbl value to 60.0


KeyError: KeyError: key "mu_b" not found

In [5]:
function set_par_dict(btc, comb_num)
    btc["par_dict"] = Dict()
    for key in btc["key_order"]
        btc["par_dict"][key] = btc["comb"][comb_num][btc["key_order"].index(key)]
    end
    
    return btc
end

MethodError: MethodError: no method matching *(::String, ::Bool)
Closest candidates are:
  *(::Any, ::Any, !Matched::Any, !Matched::Any...) at operators.jl:502
  *(!Matched::Bool, ::Bool) at bool.jl:106
  *(!Matched::Complex{Bool}, ::Bool) at complex.jl:282
  ...

In [98]:
# joinpath(pwd(), "teste_do_Artur")
# if !isdir(joinpath(pwd(), "teste_do_Artur"))
#     mkdir(joinpath(pwd(), "teste_do_Artur"))
# end